In [18]:
!pip install git+https://github.com/tagucci/pythonrouge.git 
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
from __future__ import print_function
import numpy as np
import re
import argparse
import os
import sys
import json
import time
import tensorflow as tf
import copy
import sys, math
from collections import defaultdict
from tensorflow.python.ops import rnn
import pdb
import cPickle as pickle
from tensorflow.python.ops import variable_scope
import random
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu
from os.path import basename
from pythonrouge.pythonrouge import Pythonrouge
import codecs
import platform
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu, sentence_bleu
tf.logging.set_verbosity(tf.logging.ERROR) # DEBUG, INFO, WARN, ERROR, and FATAL
# import math
class Vocab(object):
    def __init__(self, vec_path=None, dim=100, fileformat='bin',voc=None, word2id=None, word_vecs=None, unk_mapping_path=None):
        self.unk_label = 'UNK'
        self.stoplist = None
        if fileformat == 'bin':
            self.fromBinary(vec_path,voc=voc)
        elif fileformat == 'txt':
            self.fromText(vec_path,voc=voc)
        elif fileformat == 'txt2':
            self.fromText_format2(vec_path,voc=voc,pre_word_vecs=word_vecs)
        elif fileformat == 'txt3':
            self.fromText_format3(vec_path,voc=voc)
        elif fileformat == 'map':
            self.fromMap(word2id, word_vecs, word_dim=dim)
        else: # build a vocabulary with a word set
            self.fromVocabualry(voc, dim=dim)

        self.__unk_mapping = None
        if unk_mapping_path is not None:
            self.__unk_mapping = {}
            in_file = open(unk_mapping_path, 'rt')
            for line in in_file:
                items = re.split('\t', line)
                self.__unk_mapping[items[0]] = items[1]
            in_file.close()


    def get_anony_ids(self):
        self.anony_ids = set([y for x, y in self.word2id.iteritems() if re.search('_[0-9]+', x) != None])
        self.anony_toks = [x for x, y in self.word2id.iteritems() if y in self.anony_ids]
        print(self.anony_toks)

    def fromVocabualry(self, voc, dim=100):
        assert '#pad#' not in voc
        assert 'UNK' not in voc
        # load freq table and build index for each word
        self.word2id = {'#pad#':0,'UNK':1,}
        self.id2word = {0:'#pad#',1:'UNK',}

        for word in voc:
            cur_index = len(self.word2id)
            self.word2id[word] = cur_index
            self.id2word[cur_index] = word

        self.vocab_size = len(self.word2id)
        assert self.vocab_size == len(voc)+2
        self.word_dim = dim
        zero_vecs = np.zeros((1, self.word_dim), dtype=np.float32)
        shape = (self.vocab_size-1, self.word_dim)
        scale = 0.05
        normal_vecs = np.array(np.random.uniform(low=-scale, high=scale, size=shape), dtype=np.float32)
        self.word_vecs = np.concatenate((zero_vecs,normal_vecs,), axis=0)


    def fromMap(self, word2id, word_vecs, word_dim=100):
        assert False, 'not in use'
        self.word2id = word2id
        self.id2word = dict(zip(word2id.values(),word2id.keys()))

        self.vocab_size = len(word2id)
        self.word_dim = word_dim
        self.word_vecs = word_vecs


    def fromText(self, vec_path,voc=None):
        assert False, 'not in use'
        # load freq table and build index for each word
        self.word2id = {}
        self.id2word = {}

        vec_file = open(vec_path, 'rt')
        header = vec_file.readline()
        self.vocab_size, self.word_dim = map(int, header.split())
        word_vecs = {}
        for line in vec_file:
            line = line.decode('utf-8').strip()
            parts = line.split(' ')
            word = parts[0]
            if (voc is not None) and (word not in voc): continue
            vector = np.array(parts[1:], dtype='float32')
            cur_index = len(self.word2id)
            self.word2id[word] = cur_index
            self.id2word[cur_index] = word
            word_vecs[cur_index] = vector
        vec_file.close()

        self.vocab_size = len(self.word2id)
        self.word_vecs = np.zeros((self.vocab_size+1, self.word_dim), dtype=np.float32) # the last dimension is all zero
        for cur_index in xrange(self.vocab_size):
            self.word_vecs[cur_index] = word_vecs[cur_index]


    def fromText_format2(self, vec_path,voc=None,pre_word_vecs=None):
        # load freq table and build index for each word
        self.word2id = {}
        self.id2word = {}

        vec_file = open(vec_path, 'rt')
        word_vecs = {}
        for line in vec_file:
            line = line.decode('utf-8').strip()
            parts = line.split('\t')
            if len(parts) > 3:
                parts = parts[:2] + parts[-1:]
            cur_index = int(parts[0])
            word = parts[1]
            vector = np.array(map(float,re.split('\\s+', parts[2])), dtype='float32')
            assert word not in self.word2id, word
            self.word2id[word] = cur_index
            self.id2word[cur_index] = word
            word_vecs[cur_index] = vector
            self.word_dim = vector.size
        vec_file.close()

        self.vocab_size = len(self.word2id)

        if pre_word_vecs is not None:
            self.word_vecs = pre_word_vecs
        else:
            self.word_vecs = np.zeros((self.vocab_size, self.word_dim), dtype=np.float32) # the last dimension is all zero
            for cur_index in xrange(self.vocab_size):
                self.word_vecs[cur_index] = word_vecs[cur_index]


    def fromText_format3(self, vec_path,voc=None):
        assert False, 'not in use'
        # load freq table and build index for each word
        self.word2id = {}
        self.id2word = {}

        self.word2id['<s>'] = 0
        self.id2word[0] = '<s>'
        self.word2id['</s>'] = 1
        self.id2word[1] = '</s>'

        vec_file = open(vec_path, 'rt')
        word_vecs = {}
        for line in vec_file:
            line = line.decode('utf-8').strip()
            parts = line.split(' ')
            word = parts[0]
            self.word_dim = len(parts[1:])
            if (voc is not None) and (word not in voc): continue
            vector = np.array(parts[1:], dtype='float32')
            if self.word2id.has_key(word): cur_index = self.word2id[word]
            else: cur_index = len(self.word2id)
            self.word2id[word] = cur_index
            self.id2word[cur_index] = word
            word_vecs[cur_index] = vector
        vec_file.close()

        self.vocab_size = len(self.word2id)
        scale = 0.05
        self.word_vecs = np.random.uniform(low=-scale, high=scale, size=(self.vocab_size+1, self.word_dim))
        for cur_index in xrange(self.vocab_size):
            if not word_vecs.has_key(cur_index): continue
            self.word_vecs[cur_index] = word_vecs[cur_index]



    def fromText_bak(self, vec_path,voc=None):
        # load freq table and build index for each word
        self.word2id = {}
        self.id2word = {}

        vec_file = open(vec_path, 'rt')
        header = vec_file.readline()
        self.vocab_size, self.word_dim = map(int, header.split())
        self.word_vecs = np.zeros((self.vocab_size+1, self.word_dim), dtype=np.float32) # the last dimension is all zero
        for line in vec_file:
            line = line.decode('utf-8').strip()
            parts = line.split(' ')
            word = parts[0]
            if (voc is not None) and (word not in voc): continue
            vector = np.array(parts[1:], dtype='float32')
            cur_index = len(self.word2id)
            self.word2id[word] = cur_index
            self.id2word[cur_index] = word
            self.word_vecs[cur_index] = vector
        vec_file.close()

    def fromBinary_with_voc(self, fname, voc, scale=0.05, stop_num=50):
        self.stoplist = voc[0:stop_num]
        voc = voc[stop_num:]
        voc.append(self.unk_label)
        self.word2id = {}
        self.id2word = {}
        for word in voc:
            curIndex = len(self.word2id)
            self.word2id[word] = curIndex
            self.id2word[curIndex] = word

        with open(fname, "rb") as f:
            header = f.readline()
            cur_vocab_size, self.word_dim = map(int, header.split())
            word_vecs = {}
            binary_len = np.dtype('float32').itemsize * self.word_dim
            for idx in xrange(cur_vocab_size):
                word = []
                while True:
                    ch = f.read(1)
                    if ch == ' ':
                        word = ''.join(word)
                        break
                    if ch != '\n':
                        word.append(ch)
                if word in self.word2id.keys():
                    curIndex = self.word2id[word]
                else:
                    curIndex = len(self.word2id)
                    self.word2id[word] = curIndex
                    self.id2word[curIndex] = word
                word_vecs[curIndex] = np.fromstring(f.read(binary_len), dtype='float32')

        self.vocab_size = len(self.word2id)
        self.word_vecs = np.random.uniform(low=-scale, high=scale, size=(self.vocab_size+1, self.word_dim)).astype('float32')
        self.word_vecs[self.vocab_size] = self.word_vecs[self.vocab_size] * 0.0
        for cur_index in word_vecs.keys():
            self.word_vecs[cur_index] = word_vecs[cur_index]

    def fromBinary(self, fname, scale=0.05, voc=None):
        self.word2id = {}
        self.id2word = {}
        self.word2id[self.unk_label] = 0
        self.id2word[0] = self.unk_label
        # load word vector
        with open(fname, "rb") as f:
            header = f.readline()
            self.vocab_size, self.word_dim = map(int, header.split())
            word_vecs = {}
            binary_len = np.dtype('float32').itemsize * self.word_dim
            for idx in xrange(self.vocab_size):
                word = []
                while True:
                    ch = f.read(1)
                    if ch == ' ':
                        word = ''.join(word)
                        break
                    if ch != '\n':
                        word.append(ch)
                if word == '': continue
                curIndex = len(self.word2id)
                self.word2id[word] = curIndex
                self.id2word[curIndex] = word
                word_vecs[curIndex] = np.fromstring(f.read(binary_len), dtype='float32')

        # add unkwords
        if voc is not None:
            for word in voc:
                if word == '': continue
                if self.word2id.has_key(word): continue
                curIndex = len(self.word2id)
                self.word2id[word] = curIndex
                self.id2word[curIndex] = word
                word_vecs[curIndex] = np.random.uniform(low=-scale, high=scale, size=(self.word_dim,)).astype('float32')

        self.vocab_size = len(self.word2id)
        self.word_vecs = np.zeros((self.vocab_size+1, self.word_dim), dtype=np.float32) # the last dimension is all zero
        for cur_index in xrange(self.vocab_size):
            if cur_index ==0 : continue
            self.word_vecs[cur_index] = word_vecs[cur_index]
        self.word_vecs[0] = np.random.uniform(low=-scale, high=scale, size=(self.word_dim,)).astype('float32')

    def setWordvec(self,word_vecs):
        self.word_vecs = word_vecs

    def hasWord(self, word):
        return self.word2id.has_key(word)

    def size(self):
        return len(self.word2id)

    def getIndex(self, word):
        if self.stoplist is not None:
            if word in self.stoplist:
                return None
        if(self.word2id.has_key(word)):
            return self.word2id.get(word)
        else:
            return self.word2id.get('UNK')

    def getWord(self, idx):
        assert idx < self.vocab_size
        return self.id2word.get(idx)

    def getVector(self, word):
        if(self.word2id.has_key(word)):
            idx = self.word2id.get(word)
            return self.word_vecs[idx]
        return None

    def getLexical(self, sout):
        end_id = self.getIndex('</s>')
        try:
            k = sout.index(end_id)
            sout = sout[:k+1]
        except ValueError:
            pass
        slex = ' '.join([self.getWord(x) for x in sout])
        return sout, slex

    def to_index_sequence(self, sentence):
        sentence = sentence.strip()
        seq = []
        for word in re.split('\\s+', sentence):
            idx = self.getIndex(word)
            if idx == None and self.__unk_mapping is not None and self.__unk_mapping.has_key(word):
                simWord = self.__unk_mapping[word]
                idx = self.getIndex(simWord)
            if idx == None: idx = self.vocab_size
            seq.append(idx)
        return seq

    def to_index_sequence_for_list(self, words):
        seq = []
        for word in words:
            idx = self.getIndex(word)
            if idx == None and self.__unk_mapping is not None and self.__unk_mapping.has_key(word):
                simWord = self.__unk_mapping[word]
                idx = self.getIndex(simWord)
            if idx == None: idx = self.vocab_size
            seq.append(idx)
        return seq

    def to_character_matrix(self, sentence, max_char_per_word=-1):
        sentence = sentence.strip()
        return self.to_character_matrix_for_list(re.split('\\s+', sentence), max_char_per_word)

    def to_character_matrix_for_list(self, sentence, max_char_per_word=-1):
        seq = []
        for word in sentence:
            cur_seq = []
            for i in xrange(len(word)):
                cur_char = word[i]
                idx = self.getIndex(cur_char)
                if idx == None and self.__unk_mapping is not None and self.__unk_mapping.has_key(cur_char):
                    simWord = self.__unk_mapping[cur_char]
                    idx = self.getIndex(simWord)
                if idx == None: idx = self.vocab_size
                cur_seq.append(idx)
            if max_char_per_word!=-1 and len(cur_seq) > max_char_per_word:
                cur_seq = cur_seq[:max_char_per_word]
            seq.append(cur_seq)
        return seq

    def to_index_sequence4binary_features(self, sentence):
        sentence = sentence.strip().lower()
        seq = []
        for word in re.split(' ', sentence):
            idx = self.getIndex(word)
            if idx == None: continue
            seq.append(idx)
        return seq

    def to_char_ngram_index_sequence(self, sentence):
        sentence = sentence.strip().lower()
        seq = []
        words = re.split(' ', sentence)
        for word in words:
            sub_words = collect_char_ngram(word)
            for sub_word in sub_words:
                idx = self.getIndex(sub_word)
                if idx == None: continue
                seq.append(idx)
        return seq

    def to_sparse_feature_sequence(self, sentence1, sentence2):
        words1 = set(re.split(' ', sentence1.strip().lower()))
        words2 = set(re.split(' ', sentence2.strip().lower()))
        intersection_words = words1.intersection(words2)
        seq = []
        for word in intersection_words:
            idx = self.getIndex(word)
            if idx == None: continue
            seq.append(idx)
        return seq

    def get_sentence_vector(self, sentence):
        sent_vec = np.zeros((self.word_dim,), dtype='float32')
        sentence = sentence.strip().lower()
        total = 0.0
        for word in re.split(' ', sentence):
            cur_vec = self.getVector(word)
            if cur_vec is None: continue
            sent_vec += cur_vec
            total += 1.0
        if total != 0.0: sent_vec /= total
        return sent_vec

    def dump_to_txt2(self, outpath):
        outfile = open(outpath, 'wt')
        for word in self.word2id.keys():
            cur_id = self.word2id[word]
            cur_vector = self.getVector(word)
#             print(word)
            word= word.encode('utf-8')
            outline = "{}\t{}\t{}".format(cur_id, word, vec2string(cur_vector))
            outfile.write(outline + "\n")
        outfile.close()

    def dump_to_txt3(self, outpath):
        outfile = open(outpath, 'wt')
        for word in self.word2id.keys():
            cur_vector = self.getVector(word)
            word= word.encode('utf-8')
            outline = word + " {}".format(vec2string(cur_vector))
            outfile.write(outline + "\n")
        outfile.close()

def vec2string(val):
    result = ""
    for v in val:
        result += " {}".format(v)
    return result.strip()


def collect_all_ngram(words, n=2):
    all_ngrams = set()
    for i in xrange(len(words)-n):
        cur_ngram = words[i:i+n]
        all_ngrams.add(' '.join(cur_ngram))
    return all_ngrams

def collect_char_ngram(word, n=3):
    all_words = []
    if len(word)<=n: all_words.append(word)
    else:
        for i in xrange(len(word)-n+1):
            cur_word = word[i:i+3]
            all_words.append(cur_word)
    return all_words

def to_char_ngram_sequence(sentence, n=3):
    seq = []
    words = re.split(' ', sentence)
    for word in words:
        sub_words = collect_char_ngram(word)
        seq.extend(sub_words)
    return ' '.join(seq)

def collectVoc(trainpath):
    vocab = set()
    inputFile = file(trainpath, 'rt')
    for line in inputFile:
        line = line.strip()
        label, sentence = re.split('\t', line)
        sentence = sentence.lower()
        for word in re.split(' ', sentence):
            vocab.add(word)
    inputFile.close()
    return vocab

def collect_word_count(sentences, unk_num=1):
    word_count_map = {}
    for sentence in sentences:
        sentence = sentence.strip().lower()
        for word in re.split(' ', sentence):
            cur_count = 0
            if word_count_map.has_key(word):
                cur_count = word_count_map.get(word)
            word_count_map[word] = cur_count + 1
    word_count_list = []
    for word in word_count_map.keys():
        count = word_count_map.get(word)
        word_count_list.append((count, word))

    word_count_list = sorted(word_count_list,key=(lambda a:a[0]), reverse=True)
#     for i in xrange(50):
#         word, count = word_count_list[i]
#         print('{}\t{}'.format(word, count))
#     return word_count_list
    return [word for count, word in word_count_list if count>unk_num ]

def collect_word_count_with_max_vocab(sentences, max_vocab=600000):
    word_count_map = {}
    for sentence in sentences:
        sentence = sentence.strip().lower()
        for word in re.split(' ', sentence):
            cur_count = 0
            if word_count_map.has_key(word):
                cur_count = word_count_map.get(word)
            word_count_map[word] = cur_count + 1
    word_count_list = []
    for word in word_count_map.keys():
        count = word_count_map.get(word)
        word_count_list.append((count, word))

    word_count_list = sorted(word_count_list,key=(lambda a:a[0]), reverse=True)
#     for i in xrange(50):
#         word, count = word_count_list[i]
#         print('{}\t{}'.format(word, count))
#     return word_count_list
#     return [word for count, word in word_count_list if count>unk_num ]
    if len(word_count_list)<max_vocab: max_vocab = len(word_count_list)
    return [word for count, word in word_count_list[:max_vocab]]

def read_all_sentences(inpath):
    all_sentences = []
    in_file = file(inpath, 'rt')
    for line in in_file:
        if line.startswith('<'): continue
        line = line.strip().lower()
        sentences = re.split('\t', line)
        for sentence in sentences:
            sentence = sentence.strip()
            all_sentences.append(sentence)
    in_file.close()
    return all_sentences

def read_sparse_features(inpath, threshold=0.0):
    sparse_features = []
    in_file = file(inpath, 'rt')
    for line in in_file:
        line = line.strip().lower()
        items = re.split('\t', line)
        if len(items)!=2: continue
        (sparse_feature, count) = items
        count = float(count)
        if count< threshold: continue
        sparse_features.append(sparse_feature)
    in_file.close()
    return sparse_features

def build_word_index_file(word_vec_path, out_path):
    print('Loading word vectors ... ')
    vocab = Vocab(word_vec_path)
    print('Word_vecs shape: ', vocab.word_vecs.shape)
    word2id = vocab.word2id
    out_file = open(out_path,'wt')
    out_file.write('{}\t{}\n'.format(len(word2id), vocab.word_dim))
    for word in word2id.keys():
        wid = word2id[word]
        out_file.write('{}\t{}\n'.format(word, wid))
    out_file.close()
def load_word_index(index_path):
    word2id = {}
    in_file = open(index_path, 'rt')
    started = False
    for line in in_file:
        items = re.split('\t', line)
        if not started:
            started = True
            vocab_size = int(items[0])
            word_dim = int(items[1])
        else:
            if len(items)<2:
                word = ''
                word_id = int(items[0])
            else:
                word, word_id = items
            word2id[word] = int(word_id)
    in_file.close()
    return (vocab_size, word_dim, word2id)

  Cloning https://github.com/tagucci/pythonrouge.git to /tmp/pip-req-build-MnbmQC
  Running command git clone -q https://github.com/tagucci/pythonrouge.git /tmp/pip-req-build-MnbmQC
  Stored in directory: /tmp/pip-ephem-wheel-cache-AWRkFD/wheels/fd/ff/be/6716935d513fa8656ab185cb0aa70aed382b72dda42bf09c95
Successfully built pythonrouge
Mounted at /content/drive


In [0]:
def make_batches(size, batch_size):
    nb_batch = int(np.ceil(size/float(batch_size)))
    return [(i*batch_size, min(size, (i+1)*batch_size)) for i in range(0, nb_batch)] # zgwang: starting point of each batch

def pad_2d_vals_no_size(in_vals, dtype=np.int32):
    size1 = len(in_vals)
    size2 = np.max([len(x) for x in in_vals])
    return pad_2d_vals(in_vals, size1, size2, dtype=dtype)

def pad_2d_vals(in_vals, dim1_size, dim2_size, dtype=np.int32):
    out_val = np.zeros((dim1_size, dim2_size), dtype=dtype)
    if dim1_size > len(in_vals): dim1_size = len(in_vals)
    for i in xrange(dim1_size):
        cur_in_vals = in_vals[i]
        cur_dim2_size = dim2_size
        if cur_dim2_size > len(cur_in_vals): cur_dim2_size = len(cur_in_vals)
        out_val[i,:cur_dim2_size] = cur_in_vals[:cur_dim2_size]
    return out_val

def pad_3d_vals_no_size(in_vals, dtype=np.int32):
    size1 = len(in_vals)
    size2 = np.max([len(x) for x in in_vals])
    size3 = 0
    for val in in_vals:
        cur_size3 = np.max([len(x) for x in val])
        if size3<cur_size3: size3 = cur_size3
    return pad_3d_vals(in_vals, size1, size2, size3, dtype=dtype)

def pad_3d_vals(in_vals, dim1_size, dim2_size, dim3_size, dtype=np.int32):
    out_val = np.zeros((dim1_size, dim2_size, dim3_size), dtype=dtype)
    if dim1_size > len(in_vals): dim1_size = len(in_vals)
    for i in xrange(dim1_size):
        in_vals_i = in_vals[i]
        cur_dim2_size = dim2_size
        if cur_dim2_size > len(in_vals_i): cur_dim2_size = len(in_vals_i)
        for j in xrange(cur_dim2_size):
            in_vals_ij = in_vals_i[j]
            cur_dim3_size = dim3_size
            if cur_dim3_size > len(in_vals_ij): cur_dim3_size = len(in_vals_ij)
            out_val[i, j, :cur_dim3_size] = in_vals_ij[:cur_dim3_size]
    return out_val

def pad_4d_vals(in_vals, dim1_size, dim2_size, dim3_size, dim4_size, dtype=np.int32):
    out_val = np.zeros((dim1_size, dim2_size, dim3_size, dim4_size), dtype=dtype)
    if dim1_size > len(in_vals): dim1_size = len(in_vals)
    for i in xrange(dim1_size):
        in_vals_i = in_vals[i]
        cur_dim2_size = dim2_size
        if cur_dim2_size > len(in_vals_i): cur_dim2_size = len(in_vals_i)
        for j in xrange(cur_dim2_size):
            in_vals_ij = in_vals_i[j]
            cur_dim3_size = dim3_size
            if cur_dim3_size > len(in_vals_ij): cur_dim3_size = len(in_vals_ij)
            for k in xrange(cur_dim3_size):
                in_vals_ijk = in_vals_ij[k]
                cur_dim4_size = dim4_size
                if cur_dim4_size > len(in_vals_ijk): cur_dim4_size = len(in_vals_ijk)
                out_val[i, j, k, :cur_dim4_size] = in_vals_ijk[:cur_dim4_size]
    return out_val

def pad_target_labels(in_val, max_length, dtype=np.float32):
    batch_size = len(in_val)
    out_val = np.zeros((batch_size, max_length), dtype=dtype)
    for i in xrange(batch_size):
        for index in in_val[i]:
            out_val[i,index] = 1.0
    return out_val

In [0]:
class Bunch(object):
    def __init__(self, adict):
        self.__dict__.update(adict)

def save_namespace(FLAGS, out_path):
    FLAGS_dict = vars(FLAGS)
    with open(out_path, 'w') as fp:
        json.dump(FLAGS_dict, fp)
        
def load_namespace(in_path):
    with open(in_path, 'r') as fp:
        FLAGS_dict = json.load(fp)
    return Bunch(FLAGS_dict)

In [0]:
def precook(s, n=4, out=False):
    """Takes a string as input and returns an object that can be given to
    either cook_refs or cook_test. This is optional: cook_refs and cook_test
    can take string arguments as well."""
    words = s.split()
    counts = defaultdict(int)
    for k in xrange(1,n+1):
        for i in xrange(len(words)-k+1):
            ngram = tuple(words[i:i+k])
            counts[ngram] += 1
    return (len(words), counts)

def cook_refs(refs, eff=None, n=4): ## lhuang: oracle will call with "average"
    '''Takes a list of reference sentences for a single segment
    and returns an object that encapsulates everything that BLEU
    needs to know about them.'''

    reflen = []
    maxcounts = {}
    for ref in refs:
        rl, counts = precook(ref, n)
        reflen.append(rl)
        for (ngram,count) in counts.iteritems():
            maxcounts[ngram] = max(maxcounts.get(ngram,0), count)

    # Calculate effective reference sentence length.
    if eff == "shortest":
        reflen = min(reflen)
    elif eff == "average":
        reflen = float(sum(reflen))/len(reflen)

    ## lhuang: N.B.: leave reflen computaiton to the very end!!

    ## lhuang: N.B.: in case of "closest", keep a list of reflens!! (bad design)

    return (reflen, maxcounts)

def cook_test(test, (reflen, refmaxcounts), eff=None, n=4):
    '''Takes a test sentence and returns an object that
    encapsulates everything that BLEU needs to know about it.'''

    testlen, counts = precook(test, n, True)

    result = {}

    # Calculate effective reference sentence length.

    if eff == "closest":
        result["reflen"] = min((abs(l-testlen), l) for l in reflen)[1]
    else: ## i.e., "average" or "shortest" or None
        result["reflen"] = reflen

    result["testlen"] = testlen

    result["guess"] = [max(0,testlen-k+1) for k in xrange(1,n+1)]

    result['correct'] = [0]*n
    for (ngram, count) in counts.iteritems():
        result["correct"][len(ngram)-1] += min(refmaxcounts.get(ngram,0), count)

    return result

class BleuScorer(object):
    """Bleu scorer.
    """

    __slots__ = "n", "crefs", "ctest", "_score", "_ratio", "_testlen", "_reflen", "special_reflen"
    # special_reflen is used in oracle (proportional effective ref len for a node).

    def copy(self):
        ''' copy the refs.'''
        new = BleuScorer(n=self.n)
        new.ctest = copy.copy(self.ctest)
        new.crefs = copy.copy(self.crefs)
        new._score = None
        return new

    def __init__(self, test=None, refs=None, n=4, special_reflen=None):
        ''' singular instance '''

        self.n = n
        self.crefs = []
        self.ctest = []
        self.cook_append(test, refs)
        self.special_reflen = special_reflen

    def cook_append(self, test, refs):
        '''called by constructor and __iadd__ to avoid creating new instances.'''

        if refs is not None:
            self.crefs.append(cook_refs(refs))
            if test is not None:
                cooked_test = cook_test(test, self.crefs[-1])
                self.ctest.append(cooked_test) ## N.B.: -1
            else:
                self.ctest.append(None) # lens of crefs and ctest have to match

        self._score = None ## need to recompute

    def ratio(self, option=None):
        self.compute_score(option=option)
        return self._ratio

    def score_ratio(self, option=None):
        '''return (bleu, len_ratio) pair'''
        return (self.fscore(option=option), self.ratio(option=option))

    def score_ratio_str(self, option=None):
        return "%.4f (%.2f)" % self.score_ratio(option)

    def reflen(self, option=None):
        self.compute_score(option=option)
        return self._reflen

    def testlen(self, option=None):
        self.compute_score(option=option)
        return self._testlen

    def retest(self, new_test):
        if type(new_test) is str:
            new_test = [new_test]
        assert len(new_test) == len(self.crefs), new_test
        self.ctest = []
        for t, rs in zip(new_test, self.crefs):
            self.ctest.append(cook_test(t, rs))
        self._score = None

        return self

    def rescore(self, new_test):
        ''' replace test(s) with new test(s), and returns the new score.'''

        return self.retest(new_test).compute_score()

    def size(self):
        assert len(self.crefs) == len(self.ctest), "refs/test mismatch! %d<>%d" % (len(self.crefs), len(self.ctest))
        return len(self.crefs)

    def __iadd__(self, other):
        '''add an instance (e.g., from another sentence).'''

        if type(other) is tuple:
            ## avoid creating new BleuScorer instances
            self.cook_append(other[0], other[1])
        else:
            assert self.compatible(other), "incompatible BLEUs."
            self.ctest.extend(other.ctest)
            self.crefs.extend(other.crefs)
            self._score = None ## need to recompute

        return self

    def compatible(self, other):
        return isinstance(other, BleuScorer) and self.n == other.n

    def single_reflen(self, option="average"):
        return self._single_reflen(self.crefs[0][0], option)

    def _single_reflen(self, reflens, option=None, testlen=None):

        if option == "shortest":
            reflen = min(reflens)
        elif option == "average":
            reflen = float(sum(reflens))/len(reflens)
        elif option == "closest":
            reflen = min((abs(l-testlen), l) for l in reflens)[1]
        else:
            assert False, "unsupported reflen option %s" % option

        return reflen

    def recompute_score(self, option=None, verbose=0):
        self._score = None
        return self.compute_score(option, verbose)

    def compute_score(self, option=None, verbose=0):
        n = self.n
        small = 1e-9
        tiny = 1e-15 ## so that if guess is 0 still return 0
        bleu_list = [[] for _ in range(n)]

        if self._score is not None:
            return self._score

        if option is None:
            option = "average" if len(self.crefs) == 1 else "closest"

        self._testlen = 0
        self._reflen = 0
        totalcomps = {'testlen':0, 'reflen':0, 'guess':[0]*n, 'correct':[0]*n}

        # for each sentence
        for comps in self.ctest:
            testlen = comps['testlen']
            self._testlen += testlen

            if self.special_reflen is None: ## need computation
                reflen = self._single_reflen(comps['reflen'], option, testlen)
            else:
                reflen = self.special_reflen

            self._reflen += reflen

            for key in ['guess','correct']:
                for k in xrange(n):
                    totalcomps[key][k] += comps[key][k]

            # append per image bleu score
            bleu = 1.
            for k in xrange(n):
                bleu *= (float(comps['correct'][k]) + tiny) \
                        /(float(comps['guess'][k]) + small)
                bleu_list[k].append(bleu ** (1./(k+1)))
            ratio = (testlen + tiny) / (reflen + small) ## N.B.: avoid zero division
            if ratio < 1:
                for k in xrange(n):
                    bleu_list[k][-1] *= math.exp(1 - 1/ratio)

            if verbose > 1:
                print (comps, reflen)

        totalcomps['reflen'] = self._reflen
        totalcomps['testlen'] = self._testlen

        bleus = []
        bleu = 1.
        for k in xrange(n):
            bleu *= float(totalcomps['correct'][k] + tiny) \
                    / (totalcomps['guess'][k] + small)
            bleus.append(bleu ** (1./(k+1)))
        ratio = (self._testlen + tiny) / (self._reflen + small) ## N.B.: avoid zero division
        if ratio < 1:
            for k in xrange(n):
                bleus[k] *= math.exp(1 - 1/ratio)

        if verbose > 0:
            print (totalcomps)
            print ("ratio:", ratio)

        self._score = bleus
        return self._score, bleu_list

class Bleu:
    def __init__(self, n=4):
        # default compute Blue score up to 4
        self._n = n
        self._hypo_for_image = {}
        self.ref_for_image = {}

    def compute_score(self, gts, res):

        assert(gts.keys() == res.keys())
        imgIds = gts.keys()

        bleu_scorer = BleuScorer(n=self._n)
        for id in imgIds:
            hypo = res[id]
            ref = gts[id]

            # Sanity check.
            assert(type(hypo) is list)
            assert(len(hypo) == 1)
            assert(type(ref) is list)
            assert(len(ref) >= 1)

            bleu_scorer += (hypo[0], ref)

        #score, scores = bleu_scorer.compute_score(option='shortest')
        score, scores = bleu_scorer.compute_score(option='closest', verbose=0)
        #score, scores = bleu_scorer.compute_score(option='average', verbose=1)

        # return (bleu, bleu_info)
        return score, scores

    def method(self):
        return "Bleu"

In [0]:

def my_lcs(string, sub):
    """
    Calculates longest common subsequence for a pair of tokenized strings
    :param string : list of str : tokens from a string split using whitespace
    :param sub : list of str : shorter string, also split using whitespace
    :returns: length (list of int): length of the longest common subsequence between the two strings
    Note: my_lcs only gives length of the longest common subsequence, not the actual LCS
    """
    if(len(string)< len(sub)):
        sub, string = string, sub

    lengths = [[0 for i in range(0,len(sub)+1)] for j in range(0,len(string)+1)]

    for j in range(1,len(sub)+1):
        for i in range(1,len(string)+1):
            if(string[i-1] == sub[j-1]):
                lengths[i][j] = lengths[i-1][j-1] + 1
            else:
                lengths[i][j] = max(lengths[i-1][j] , lengths[i][j-1])

    return lengths[len(string)][len(sub)]

class Rouge():
    '''
    Class for computing ROUGE-L score for a set of candidate sentences for the MS COCO test set
    '''
    def __init__(self):
        # vrama91: updated the value below based on discussion with Hovey
        self.beta = 1.2

    def calc_score(self, candidate, refs):
        """
        Compute ROUGE-L score given one candidate and references for an image
        :param candidate: str : candidate sentence to be evaluated
        :param refs: list of str : COCO reference sentences for the particular image to be evaluated
        :returns score: int (ROUGE-L score for the candidate evaluated against references)
        """
        assert(len(candidate)==1)    
        assert(len(refs)>0)         
        prec = []
        rec = []

        # split into tokens
        token_c = candidate[0].split(" ")
        
        for reference in refs:
            # split into tokens
            token_r = reference.split(" ")
            # compute the longest common subsequence
            lcs = my_lcs(token_r, token_c)
            prec.append(lcs/float(len(token_c)))
            rec.append(lcs/float(len(token_r)))

        prec_max = max(prec)
        rec_max = max(rec)

        if(prec_max!=0 and rec_max !=0):
            score = ((1 + self.beta**2)*prec_max*rec_max)/float(rec_max + self.beta**2*prec_max)
        else:
            score = 0.0
        return score

    def compute_score(self, gts, res):
        """
        Computes Rouge-L score given a set of reference and candidate sentences for the dataset
        Invoked by evaluate_captions.py 
        :param hypo_for_image: dict : candidate / test sentences with "image name" key and "tokenized sentences" as values 
        :param ref_for_image: dict : reference MS-COCO sentences with "image name" key and "tokenized sentences" as values
        :returns: average_score: float (mean ROUGE-L score computed by averaging scores for all the images)
        """
        assert(gts.keys() == res.keys())
        imgIds = gts.keys()

        score = []
        for id in imgIds:
            hypo = res[id]
            ref  = gts[id]

            score.append(self.calc_score(hypo, ref))

            # Sanity check.
            assert(type(hypo) is list)
            assert(len(hypo) == 1)
            assert(type(ref) is list)
            assert(len(ref) > 0)

        average_score = np.mean(np.array(score))
        return average_score, np.array(score)

    def method(self):
        return "Rouge"

In [0]:
def score_all(ref, hypo):
    scorers = [
        (Bleu(4),["Bleu_1","Bleu_2","Bleu_3","Bleu_4"]),
        (Rouge(),"ROUGE_L"),
    ]
    final_scores = {}
    for scorer,method in scorers:
        score,scores = scorer.compute_score(ref,hypo)
        if type(score)==list:
            for m,s in zip(method,score):
                final_scores[m] = s
        else:
            final_scores[method] = score

    return final_scores

def score(ref, hypo):
    scorers = [
        (Bleu(4),["Bleu_1","Bleu_2","Bleu_3","Bleu_4"])

    ]
    final_scores = {}
    for scorer,method in scorers:
        score,scores = scorer.compute_score(ref,hypo)
        if type(score)==list:
            for m,s in zip(method,score):
                final_scores[m] = s
        else:
            final_scores[method] = score

    return final_scores

def evaluate_captions(ref,cand):
    hypo = {}
    refe = {}
    for i, caption in enumerate(cand):
        hypo[i] = [caption,]
        refe[i] = ref[i]
    final_scores = score(refe, hypo)
    return 1*final_scores['Bleu_4'] + 1*final_scores['Bleu_3'] + 0.5*final_scores['Bleu_1'] + 0.5*final_scores['Bleu_2']

def evaluate(data_path='./data', split='val', get_scores=False):
    reference_path = os.path.join(data_path, "%s/%s.references.pkl" %(split, split))
    candidate_path = os.path.join(data_path, "%s/%s.candidate.captions.pkl" %(split, split))

    # load caption data
    with open(reference_path, 'rb') as f:
        ref = pickle.load(f)
    with open(candidate_path, 'rb') as f:
        cand = pickle.load(f)

    # make dictionary
    hypo = {}
    for i, caption in enumerate(cand):
        hypo[i] = [caption]

    # compute bleu score
    final_scores = score_all(ref, hypo)

    # print out scores
    print ('Bleu_1:\t',final_scores['Bleu_1'])
    print ('Bleu_2:\t',final_scores['Bleu_2'])
    print ('Bleu_3:\t',final_scores['Bleu_3'])
    print ('Bleu_4:\t',final_scores['Bleu_4'])
    print ('METEOR:\t',final_scores['METEOR'])
    print ('ROUGE_L:',final_scores['ROUGE_L'])
    print ('CIDEr:\t',final_scores['CIDEr'])

    if get_scores:
        return final_scores

In [0]:
eps = 1e-6
def cosine_distance(y1,y2):
    # y1 [....,a, 1, d]
    # y2 [....,1, b, d]
    cosine_numerator = tf.reduce_sum(tf.multiply(y1, y2), axis=-1)
    y1_norm = tf.sqrt(tf.maximum(tf.reduce_sum(tf.square(y1), axis=-1), eps))
    y2_norm = tf.sqrt(tf.maximum(tf.reduce_sum(tf.square(y2), axis=-1), eps))
    return cosine_numerator / y1_norm / y2_norm


def euclidean_distance(y1,y2):
    distance = tf.sqrt(tf.maximum(tf.reduce_sum(tf.square(y1 - y2), axis=-1), eps))
    return distance

def match_same_row_matrix(matrix1, matrix2, feature_dim, options):
    # matrix1: [num_rows, feature_dim]
    # matrix2: [num_rows, feature_dim]
    input_shape = tf.shape(matrix1)
    num_rows = input_shape[0]
    matching_result = []
    matching_dim = 0
    if options.has_key('cosine'):
        cosine_value = cosine_distance(matrix1, matrix2)
        cosine_value = tf.reshape(cosine_value, [num_rows, 1])
        matching_result.append(cosine_value)
        matching_dim += 1

    if options.has_key('euclidean'):
        euclidean_value = euclidean_distance(matrix1, matrix2)
        euclidean_value = tf.reshape(euclidean_value, [num_rows, 1])
        matching_result.append(euclidean_value)
        matching_dim += 1

    if options.has_key('subtract'):
        cur_matching = matrix1-matrix2
#         cur_matching = tf.reshape(cur_matching, [num_rows, feature_dim])
        matching_result.append(cur_matching)
        matching_dim += feature_dim

    if options.has_key('multiply'):
        cur_matching = matrix1*matrix2
#         cur_matching = tf.reshape(cur_matching, [num_rows, feature_dim])
        matching_result.append(cur_matching)
        matching_dim += feature_dim

    if options.has_key('nn'):
        (nn_dim, w, b) = options['nn']
#         with tf.variable_scope(name_scope + "-nn"):
#             w = tf.get_variable("w", [2*feature_dim, nn_dim], dtype=tf.float32)
#             b = tf.get_variable("b", [nn_dim], dtype=tf.float32)
        NN_input = tf.concat(1, [matrix1, matrix2])
        NN_input = tf.reshape(NN_input, [num_rows, 2*feature_dim])
        NN_matching = tf.tanh(tf.matmul(NN_input, w) + b)
        NN_matching = tf.reshape(NN_matching, [num_rows, nn_dim])
        matching_result.append(NN_matching)
        matching_dim += nn_dim

    matrix1_tmp = tf.expand_dims(matrix1, axis=1) #[num_rows, 'x', feature_dim]
    matrix2_tmp = tf.expand_dims(matrix2, axis=1) #[num_rows, 'x', feature_dim]
    if options.has_key('mp-cosine'):
        (cosine_MP_dim, mp_cosine_params) = options['mp-cosine']
#         mp_cosine_params = tf.get_variable(name_scope + "-mp-cosine", shape=[cosine_MP_dim, feature_dim], dtype=tf.float32)
        mp_cosine_params_tmp = tf.expand_dims(mp_cosine_params, axis=0) # ['x', cosine_MP_dim, feature_dim]
        mp_cosine_matching = cosine_distance(tf.multiply(matrix1_tmp, mp_cosine_params_tmp), tf.multiply(matrix2_tmp, mp_cosine_params_tmp))
#         mp_cosine_matching = tf.reshape(mp_cosine_matching, [num_rows, cosine_MP_dim])
        matching_result.append(mp_cosine_matching)
        matching_dim += cosine_MP_dim

    if options.has_key('mp-euclidean'):
        (euclidean_MP_dim, mp_euclidean_params) = options['mp-euclidean']
#         mp_euclidean_params = tf.get_variable(name_scope + "-mp-euclidean", shape=[euclidean_MP_dim, feature_dim], dtype=tf.float32)
        mp_euclidean_params_tmp = tf.expand_dims(mp_euclidean_params, axis=0) # ['x', euclidean_MP_dim, feature_dim]
        mp_euclidean_matching = euclidean_distance(tf.multiply(matrix1_tmp, mp_euclidean_params_tmp), tf.multiply(matrix2_tmp, mp_euclidean_params_tmp))
#         mp_euclidean_matching = tf.reshape(mp_euclidean_matching, [num_rows, euclidean_MP_dim])
        matching_result.append(mp_euclidean_matching)
        matching_dim += euclidean_MP_dim

    matching_result = tf.concat(1, matching_result)
    return (matching_result, matching_dim)

def match_matrix_bak(matrix1, matrix2, feature_dim, options):
    # matrix1: [num_rows1, feature_dim]
    # matrix2: [num_rows2, feature_dim]
    num_rows1 = tf.shape(matrix1)[0]
    num_rows2 = tf.shape(matrix2)[0]

    matrix1_tmp = tf.expand_dims(matrix1, axis=1) # [num_rows1, 'x', feature_dim]
    matrix1_tmp = tf.tile(matrix1_tmp, [1, num_rows2, 1], name=None)# [num_rows1, num_rows2, feature_dim]
    matrix1_tmp = tf.reshape(matrix1_tmp, [num_rows1*num_rows2, feature_dim])

    matrix2_tmp = tf.expand_dims(matrix2, axis=0) # ['x', num_rows1, feature_dim]
    matrix2_tmp = tf.tile(matrix2_tmp, [num_rows1, 1, 1], name=None)# [num_rows1, num_rows2, feature_dim]
    matrix2_tmp = tf.reshape(matrix2_tmp, [num_rows1*num_rows2, feature_dim])

    (matching_result, matching_dim) = match_same_row_matrix(matrix1_tmp, matrix2_tmp, feature_dim, options)
    matching_result = tf.reshape(matching_result, [num_rows1, num_rows2, matching_dim])
    return (matching_result, matching_dim)

def match_matrix_bak2(matrix1, matrix2, feature_dim, options):
    # matrix1: [num_rows1, feature_dim]
    # matrix2: [num_rows2, feature_dim]
    num_rows2 = tf.shape(matrix2)[0]
    def singel_instance(x):
        # x: [feature_dim]
        x = tf.reshape(x, [1, feature_dim]) # ['x', feature_dim]
        x = tf.tile(x, [num_rows2, 1])# [num_rows2, feature_dim]
        (cur_matching_result, _) = match_same_row_matrix(x, matrix2, feature_dim, options)
        return cur_matching_result
    matching_result = tf.map_fn(singel_instance, matrix1, dtype=tf.float32) # [num_rows1, num_rows2, matching_dim]
    matching_dim = options['matching_dim']
    return (matching_result, matching_dim)

def match_matrix(matrix1, matrix2, feature_dim, options):
    # matrix1: [num_rows1, feature_dim]
    # matrix2: [num_rows2, feature_dim]
    num_rows1 = tf.shape(matrix1)[0]
    num_rows2 = tf.shape(matrix2)[0]
    def singel_instance(x):
        # x: [feature_dim]
        x = tf.reshape(x, [1, feature_dim]) # ['x', feature_dim]
        def single_word(y):
            # y: {features_dim}
            y = tf.reshape(y, [1, feature_dim]) # ['x', feature_dim]
            (cur_matching_result, _) = match_same_row_matrix(x, y, feature_dim, options)
            return cur_matching_result
        return tf.map_fn(single_word, matrix2, dtype=tf.float32) # [num_rows1, num_rows2, matching_dim]
    matching_dim = options['matching_dim']
    matching_result = tf.map_fn(singel_instance, matrix1, dtype=tf.float32) # [num_rows1, num_rows2, matching_dim]
    matching_result = tf.reshape(matching_result, [num_rows1, num_rows2, matching_dim])
    return (matching_result, matching_dim)


def create_matching_params(feature_dim, options, name_scope):
    options_with_params = {}
    matching_dim = 0
    if options.with_cosine:
        options_with_params['cosine'] = 'cosine'
        matching_dim += 1

    if options.with_euclidean:
        options_with_params['euclidean'] = 'euclidean'
        matching_dim += 1

    if options.with_subtract:
        options_with_params['subtract'] = 'subtract'
        matching_dim += feature_dim

    if options.with_multiply:
        options_with_params['multiply'] = 'multiply'
        matching_dim += feature_dim

    if options.with_nn_match:
        nn_dim = options.nn_match_dim
        with tf.variable_scope(name_scope + "-nn"):
            w = tf.get_variable("w_nn_match", [2*feature_dim, nn_dim], dtype=tf.float32)
            b = tf.get_variable("b_nn_match", [nn_dim], dtype=tf.float32)
        options_with_params['nn'] = (nn_dim, w, b)
        matching_dim += nn_dim

    if options.with_mp_cosine:
        cosine_MP_dim = options.cosine_MP_dim
        mp_cosine_params = tf.get_variable(name_scope + "-mp-cosine", shape=[cosine_MP_dim, feature_dim], dtype=tf.float32)
        options_with_params['mp-cosine'] = (cosine_MP_dim, mp_cosine_params)
        matching_dim += cosine_MP_dim

    if options.with_mp_euclidean:
        euclidean_MP_dim = options.euclidean_MP_dim
        mp_euclidean_params = tf.get_variable(name_scope + "-mp-euclidean", shape=[euclidean_MP_dim, feature_dim], dtype=tf.float32)
        options_with_params['mp-euclidean'] = (euclidean_MP_dim, mp_euclidean_params)
        matching_dim += euclidean_MP_dim
    options_with_params['matching_dim'] = matching_dim
    return options_with_params

def calculate_full_matching(passage_representation, full_question_representation, feature_dim, options, name_scope):
    # passage_representation: [batch_size, passage_len, feature_dim]
    # full_question_representation: [batch_size, feature_dim]

    # create parameters
    options_with_params = create_matching_params(feature_dim, options, name_scope)
    matching_dim = options_with_params['matching_dim']

    in_shape = tf.shape(passage_representation)
    batch_size = in_shape[0]
    passage_len = in_shape[1]

    def singel_instance(x):
        p = x[0]
        q = x[1]
        # p: [pasasge_len, feature_dim], q: [feature_dim]
        q = tf.expand_dims(q, axis=0) # ['x', feature_dim]
        q = tf.tile(q, [passage_len, 1])# [pasasge_len, feature_dim]
        (cur_matching_result, _) = match_same_row_matrix(p, q, feature_dim, options_with_params)
        return cur_matching_result
    elems = (passage_representation, full_question_representation)
    matching_result = tf.map_fn(singel_instance, elems, dtype=tf.float32) # [batch_size, passage_len, decompse_dim]
#     matching_result = tf.reshape(matching_result, [batch_size, passage_len, matching_dim])
    return (matching_result, matching_dim)

def calculate_maxpooling_matching(passage_rep, question_rep, feature_dim, options, name_scope):
    # passage_representation: [batch_size, passage_len, dim]
    # qusetion_representation: [batch_size, question_len, dim]

    # create parameters
    options_with_params = create_matching_params(feature_dim, options, name_scope)
    matching_dim = options_with_params['matching_dim']

    in_shape = tf.shape(passage_rep)
    batch_size = in_shape[0]
    passage_len = in_shape[1]

    def singel_instance(x):
        p = x[0] # p: [pasasge_len, dim]
        q = x[1] # q: [question_len, dim]
        (cur_matching_result, _) = match_matrix(p, q, feature_dim, options_with_params)
        return cur_matching_result # [pasasge_len, question_len, matching_dim]
    elems = (passage_rep, question_rep)
    matching_result = tf.map_fn(singel_instance, elems, dtype=tf.float32) # [batch_size, passage_len, question_len, matching_dim]
    matching_result = tf.concat(2, [tf.reduce_max(matching_result, axis=2), tf.reduce_mean(matching_result, axis=2)])
#     matching_result = tf.reshape(matching_result, [batch_size, passage_len, 2*matching_dim])
    return (matching_result, 2*matching_dim)

def calculate_attentive_matching(passage_rep, att_question_rep, feature_dim, options, name_scope):
    # passage_rep: [batch_size, passage_len, dim]
    # att_question_rep: [batch_size, passage_len, dim]

    # create parameters
    options_with_params = create_matching_params(feature_dim, options, name_scope)
    matching_dim = options_with_params['matching_dim']

    in_shape = tf.shape(passage_rep)
    batch_size = in_shape[0]
    passage_len = in_shape[1]

    def singel_instance(x):
        p = x[0] # p: [pasasge_len, dim]
        q = x[1] # q: [pasasge_len, dim]
        (cur_matching_result, _) = match_same_row_matrix(p, q, feature_dim, options_with_params)
        return cur_matching_result

    elems = (passage_rep, att_question_rep)
    matching_result = tf.map_fn(singel_instance, elems, dtype=tf.float32)
#     matching_result = tf.reshape(matching_result, [batch_size, passage_len, matching_dim])
    return (matching_result, matching_dim)

def calculate_cosine_weighted_question_representation(question_representation, cosine_matrix, normalize=False):
    # question_representation: [batch_size, question_len, dim]
    # cosine_matrix: [batch_size, passage_len, question_len]
    if normalize: cosine_matrix = tf.nn.softmax(cosine_matrix)
    expanded_cosine_matrix = tf.expand_dims(cosine_matrix, axis=-1) # [batch_size, passage_len, question_len, 'x']
    weighted_question_words = tf.expand_dims(question_representation, axis=1) # [batch_size, 'x', question_len, dim]
    weighted_question_words = tf.reduce_sum(tf.multiply(weighted_question_words, expanded_cosine_matrix), axis=2)# [batch_size, passage_len, dim]
    if not normalize:
        weighted_question_words = tf.div(weighted_question_words, tf.expand_dims(tf.add(tf.reduce_sum(cosine_matrix, axis=-1),eps),axis=-1))
    return weighted_question_words # [batch_size, passage_len, dim]

def calculate_max_question_representation(question_representation, cosine_matrix):
    # question_representation: [batch_size, question_len, dim]
    # cosine_matrix: [batch_size, passage_len, question_len]
    question_index = tf.arg_max(cosine_matrix, 2) # [batch_size, passage_len]
    def singel_instance(x):
        q = x[0]
        c = x[1]
        return tf.gather(q, c)
    elems = (question_representation, question_index)
    return tf.map_fn(singel_instance, elems, dtype=tf.float32) # [batch_size, passage_len, decompse_dim]

def calculate_local_question_representation(question_representation, cosine_matrix, win_size):
    # question_representation: [batch_size, question_len, dim]
    # cosine_matrix: [batch_size, passage_len, question_len]
    in_shape = tf.shape(question_representation)
#     batch_size = in_shape[0]
    question_len = tf.cast(in_shape[1], tf.int64)
    question_index = tf.arg_max(cosine_matrix, 2) # [batch_size, passage_len]
    def singel_instance(x):
        q = x[0] # question_representation: [question_len, dim]
        c = x[1] # question_index: [question_len]
        result = tf.gather(q, c)
        for i in xrange(win_size):
            cur_index = tf.subtract(c, i+1)
            cur_index = tf.maximum(cur_index, 0)
            result = result + tf.gather(q, cur_index)

        for i in xrange(1, win_size):
            cur_index = tf.add(c, i+1)
            cur_index = tf.minimum(cur_index, question_len-1)
            result = result + tf.gather(q, cur_index)

        return result / (2*win_size + 1)
    elems = (question_representation, question_index)
    return tf.map_fn(singel_instance, elems, dtype=tf.float32) # [batch_size, passage_len, decompse_dim]

def cal_linear_decomposition_representation(passage_representation, passage_lengths, cosine_matrix,is_training,
                                            lex_decompsition_dim, dropout_rate):
    # passage_representation: [batch_size, passage_len, dim]
    # cosine_matrix: [batch_size, passage_len, question_len]
    passage_similarity = tf.reduce_max(cosine_matrix, 2)# [batch_size, passage_len]
    similar_weights = tf.expand_dims(passage_similarity, -1) # [batch_size, passage_len, 1]
    dissimilar_weights = tf.subtract(1.0, similar_weights)
    similar_component = tf.multiply(passage_representation, similar_weights)
    dissimilar_component = tf.multiply(passage_representation, dissimilar_weights)
    all_component = tf.concat(2, [similar_component, dissimilar_component])
    if lex_decompsition_dim==-1:
        return all_component
    with tf.variable_scope('lex_decomposition'):
        lex_lstm_cell_fw = tf.nn.rnn_cell.BasicLSTMCell(lex_decompsition_dim)
        lex_lstm_cell_bw = tf.nn.rnn_cell.BasicLSTMCell(lex_decompsition_dim)
        if is_training:
            lex_lstm_cell_fw = tf.nn.rnn_cell.DropoutWrapper(lex_lstm_cell_fw, output_keep_prob=(1 - dropout_rate))
            lex_lstm_cell_bw = tf.nn.rnn_cell.DropoutWrapper(lex_lstm_cell_bw, output_keep_prob=(1 - dropout_rate))
        lex_lstm_cell_fw = tf.nn.rnn_cell.MultiRNNCell([lex_lstm_cell_fw])
        lex_lstm_cell_bw = tf.nn.rnn_cell.MultiRNNCell([lex_lstm_cell_bw])

        (lex_features_fw, lex_features_bw), _ = rnn.bidirectional_dynamic_rnn(
                    lex_lstm_cell_fw, lex_lstm_cell_bw, all_component, dtype=tf.float32, sequence_length=passage_lengths)

        lex_features = tf.concat(2, [lex_features_fw, lex_features_bw])
    return lex_features



def cal_relevancy_matrix(in_question_repres, in_passage_repres):
    in_question_repres_tmp = tf.expand_dims(in_question_repres, 1) # [batch_size, 1, question_len, dim]
    in_passage_repres_tmp = tf.expand_dims(in_passage_repres, 2) # [batch_size, passage_len, 1, dim]
    relevancy_matrix = cosine_distance(in_question_repres_tmp,in_passage_repres_tmp) # [batch_size, passage_len, question_len]
    return relevancy_matrix

def mask_relevancy_matrix(relevancy_matrix, question_mask, passage_mask):
    # relevancy_matrix: [batch_size, passage_len, question_len]
    # question_mask: [batch_size, question_len]
    # passage_mask: [batch_size, passsage_len]
    if question_mask is not None:
        relevancy_matrix = tf.multiply(relevancy_matrix, tf.expand_dims(question_mask, 1))
    relevancy_matrix = tf.multiply(relevancy_matrix, tf.expand_dims(passage_mask, 2))
    return relevancy_matrix

def match_passage_with_question(passage_context_representation_fw, passage_context_representation_bw, mask,
                                question_context_representation_fw, question_context_representation_bw,question_mask,
                                context_lstm_dim, with_full_matching=True, with_attentive_matching=True,
                                with_max_attentive_matching=True, with_maxpooling_matching=True,
                                with_forward_match=True, with_backward_match=True, match_options=None,
                                with_local_attentive_matching=False, win_size=3):
    all_question_aware_representatins = []
    dim = 0

    if with_forward_match:
        if question_mask is not None:
            question_context_representation_fw = tf.multiply(question_context_representation_fw, tf.expand_dims(question_mask,-1))
        passage_context_representation_fw = tf.multiply(passage_context_representation_fw, tf.expand_dims(mask,-1))

    if with_backward_match:
        if question_mask is not None:
            question_context_representation_bw = tf.multiply(question_context_representation_bw, tf.expand_dims(question_mask,-1))
        passage_context_representation_bw = tf.multiply(passage_context_representation_bw, tf.expand_dims(mask,-1))

    if with_full_matching:
        # forward full matching
        if with_forward_match:
            fw_question_full_rep = question_context_representation_fw[:,-1,:]
            (fw_full_match_rep, matching_dim) = calculate_full_matching(passage_context_representation_fw, fw_question_full_rep,
                                                                         context_lstm_dim, match_options, 'fw_full_match')
            all_question_aware_representatins.append(fw_full_match_rep)
            dim += matching_dim

        # backward full matching
        if with_backward_match:
            bw_question_full_rep = question_context_representation_bw[:,0,:]
            (bw_full_match_rep, matching_dim) = calculate_full_matching(passage_context_representation_bw, bw_question_full_rep,
                                                                         context_lstm_dim, match_options, 'bw_full_match')
            all_question_aware_representatins.append(bw_full_match_rep)
            dim += matching_dim

    if with_maxpooling_matching:
        # forward Maxpooling-Matching
        if with_forward_match:
            (fw_maxpooling_rep, matching_dim) = calculate_maxpooling_matching(passage_context_representation_fw,
                                        question_context_representation_fw, context_lstm_dim, match_options, 'fw_maxpooling_match')
            all_question_aware_representatins.append(fw_maxpooling_rep)
            dim += matching_dim

        # backward Maxpooling-Matching
        if with_backward_match:
            (bw_maxpooling_rep, matching_dim) = calculate_maxpooling_matching(passage_context_representation_bw,
                                    question_context_representation_bw, context_lstm_dim, match_options, 'bw_maxpooling_match')
            all_question_aware_representatins.append(bw_maxpooling_rep)
            dim += matching_dim

    if with_forward_match:
        forward_relevancy_matrix = cal_relevancy_matrix(question_context_representation_fw, passage_context_representation_fw)
        forward_relevancy_matrix = mask_relevancy_matrix(forward_relevancy_matrix, question_mask, mask)

    if with_backward_match:
        backward_relevancy_matrix = cal_relevancy_matrix(question_context_representation_bw, passage_context_representation_bw)
        backward_relevancy_matrix = mask_relevancy_matrix(backward_relevancy_matrix, question_mask, mask)

    if with_attentive_matching:
        # forward attentive-matching
        if with_forward_match:
            att_question_fw_contexts = calculate_cosine_weighted_question_representation(question_context_representation_fw,
                                                                                     forward_relevancy_matrix)
            (fw_attentive_rep, matching_dim) = calculate_attentive_matching(passage_context_representation_fw,
                                                    att_question_fw_contexts, context_lstm_dim, match_options, 'fw_attentive_match')
            all_question_aware_representatins.append(fw_attentive_rep)
            dim += matching_dim

        # backward attentive-matching
        if with_backward_match:
            att_question_bw_contexts = calculate_cosine_weighted_question_representation(question_context_representation_bw,
                                                                                     backward_relevancy_matrix)
            (bw_attentive_rep, matching_dim) = calculate_attentive_matching(passage_context_representation_bw,
                                                    att_question_bw_contexts, context_lstm_dim, match_options, 'bw_attentive_match')
            all_question_aware_representatins.append(bw_attentive_rep)
            dim += matching_dim

    if with_max_attentive_matching:
        # forward max attentive-matching
        if with_forward_match:
            max_att_fw = calculate_max_question_representation(question_context_representation_fw, forward_relevancy_matrix)
            (fw_max_attentive_rep, matching_dim) = calculate_attentive_matching(passage_context_representation_fw,
                                                    max_att_fw, context_lstm_dim, match_options, 'fw_max_attentive_match')
            all_question_aware_representatins.append(fw_max_attentive_rep)
            dim += matching_dim

        # backward max attentive-matching
        if with_backward_match:
            max_att_bw = calculate_max_question_representation(question_context_representation_bw, backward_relevancy_matrix)
            (bw_max_attentive_rep, matching_dim) = calculate_attentive_matching(passage_context_representation_bw,
                                                    max_att_bw, context_lstm_dim, match_options, 'bw_max_attentive_match')
            all_question_aware_representatins.append(bw_max_attentive_rep)
            dim += matching_dim

    if with_local_attentive_matching:
        # forward max attentive-matching
        if with_forward_match:
            local_att_fw = calculate_local_question_representation(question_context_representation_fw, forward_relevancy_matrix, win_size)
            (fw_local_attentive_rep, matching_dim) = calculate_attentive_matching(passage_context_representation_fw,
                                                    local_att_fw, context_lstm_dim, match_options, 'fw_local_attentive_match')
            all_question_aware_representatins.append(fw_local_attentive_rep)
            dim += matching_dim

        # backward max attentive-matching
        if with_backward_match:
            local_att_bw = calculate_local_question_representation(question_context_representation_bw, backward_relevancy_matrix, win_size)
            (bw_local_attentive_rep, matching_dim) = calculate_attentive_matching(passage_context_representation_bw,
                                                    local_att_bw, context_lstm_dim, match_options, 'bw_local_attentive_match')
            all_question_aware_representatins.append(bw_local_attentive_rep)
            dim += matching_dim

    if with_forward_match:
        all_question_aware_representatins.append(tf.reduce_max(forward_relevancy_matrix, axis=2,keep_dims=True))
        all_question_aware_representatins.append(tf.reduce_mean(forward_relevancy_matrix, axis=2,keep_dims=True))
        dim += 2

    if with_backward_match:
        all_question_aware_representatins.append(tf.reduce_max(backward_relevancy_matrix, axis=2,keep_dims=True))
        all_question_aware_representatins.append(tf.reduce_mean(backward_relevancy_matrix, axis=2,keep_dims=True))
        dim += 2
    return (all_question_aware_representatins, dim)

def cross_entropy(logits, truth, mask):
    # logits: [batch_size, passage_len]
    # truth: [batch_size, passage_len]
    # mask: [batch_size, passage_len]

#     xdev = x - x.max()
#     return xdev - T.log(T.sum(T.exp(xdev)))
    logits = tf.multiply(logits, mask)
    xdev = tf.subtract(logits, tf.expand_dims(tf.reduce_max(logits, 1), -1))
    log_predictions = tf.subtract(xdev, tf.expand_dims(tf.log(tf.reduce_sum(tf.exp(xdev),-1)),-1))
#     return -T.sum(targets * log_predictions)
    result = tf.multiply(tf.multiply(truth, log_predictions), mask) # [batch_size, passage_len]
    return tf.multiply(-1.0,tf.reduce_sum(result, -1)) # [batch_size]

def highway_layer(in_val, output_size, scope=None):
    # in_val: [batch_size, passage_len, dim]
    input_shape = tf.shape(in_val)
    batch_size = input_shape[0]
    passage_len = input_shape[1]
#     feat_dim = input_shape[2]
    in_val = tf.reshape(in_val, [batch_size * passage_len, output_size])
    with tf.variable_scope(scope or "highway_layer"):
        highway_w = tf.get_variable("highway_w", [output_size, output_size], dtype=tf.float32)
        highway_b = tf.get_variable("highway_b", [output_size], dtype=tf.float32)
        full_w = tf.get_variable("full_w", [output_size, output_size], dtype=tf.float32)
        full_b = tf.get_variable("full_b", [output_size], dtype=tf.float32)
        trans = tf.nn.tanh(tf.nn.xw_plus_b(in_val, full_w, full_b))
        gate = tf.nn.sigmoid(tf.nn.xw_plus_b(in_val, highway_w, highway_b))
        outputs = tf.add(tf.multiply(trans, gate), tf.multiply(in_val, tf.subtract(1.0, gate)), "y")
    outputs = tf.reshape(outputs, [batch_size, passage_len, output_size])
    return outputs

def multi_highway_layer(in_val, output_size, num_layers, scope=None):
    scope_name = 'highway_layer'
    if scope is not None: scope_name = scope
    for i in xrange(num_layers):
        cur_scope_name = scope_name + "-{}".format(i)
        in_val = highway_layer(in_val, output_size, scope=cur_scope_name)
    return in_val



In [0]:
def collect_neighbor_node_representations(representation, positions):
    # representation: [batch_size, num_nodes, feature_dim]
    # positions: [batch_size, num_nodes, num_neighbors]
    feature_dim = tf.shape(representation)[2]
    input_shape = tf.shape(positions)
    batch_size = input_shape[0]
    num_nodes = input_shape[1]
    num_neighbors = input_shape[2]
    positions_flat = tf.reshape(positions, [batch_size, num_nodes*num_neighbors])
    def singel_instance(x):
        # x[0]: [num_nodes, feature_dim]
        # x[1]: [num_nodes*num_neighbors]
        return tf.gather(x[0], x[1])
    elems = (representation, positions_flat)
    representations = tf.map_fn(singel_instance, elems, dtype=tf.float32)
    return tf.reshape(representations, [batch_size, num_nodes, num_neighbors, feature_dim])

def collect_node_representations(representation, positions):
    # representation: [batch_size, num_nodes, feature_dim]
    # positions: [batch_size, num_candidate_nodes]
    def singel_instance(x):
        # x[0]: [num_nodes, feature_dim]
        # x[1]: [num_candidate_nodes]
        return tf.gather(x[0], x[1])
    elems = (representation, positions)
    return tf.map_fn(singel_instance, elems, dtype=tf.float32) # [batch_size, num_candidate_nodes, feature_dim]


def graph_match(in_question_repres, in_passage_repres, 
                question_mask, passage_mask, edge_embedding,
                question_neighbor_indices, passage_neighbor_indices,
                question_neighbor_edges, passage_neighbor_edges,
                question_neighbor_size, passage_neighbor_size,
                neighbor_vector_dim, input_dim, edge_dim, num_syntax_match_layer,
                with_attentive_matching=True, with_max_attentive_matching=True,
                with_maxpooling_matching=True, match_options=None): 

    all_matching_vectors = []
    all_matching_dim = 0
    
    
    input_shape = tf.shape(question_neighbor_indices)
    batch_size = input_shape[0]
    question_len = input_shape[1]
    num_question_neighbors = input_shape[2]

    input_shape = tf.shape(passage_neighbor_indices)
#     batch_size = input_shape[0]
    passage_len = input_shape[1]
    num_passage_neighbors = input_shape[2]
    
    question_neighbor_mask = tf.sequence_mask(tf.reshape(question_neighbor_size, [-1]), num_question_neighbors, dtype=tf.float32)
    question_neighbor_mask = tf.reshape(question_neighbor_mask, [batch_size, question_len, num_question_neighbors])

    passage_neighbor_mask = tf.sequence_mask(tf.reshape(passage_neighbor_size, [-1]), num_passage_neighbors, dtype=tf.float32)
    passage_neighbor_mask = tf.reshape(passage_neighbor_mask, [batch_size, passage_len, num_passage_neighbors])

    question_neighbor_edge_representations = tf.nn.embedding_lookup(edge_embedding, question_neighbor_edges) 
    # [batch_size, question_len, num_question_neighbors, edge_dim]
    passage_neighbor_edge_representations = tf.nn.embedding_lookup(edge_embedding, passage_neighbor_edges) 
    # [batch_size, passage_len, num_passage_neighbors, edge_dim]
    question_neighbor_node_representations = collect_neighbor_node_representations(
                                                    in_question_repres, question_neighbor_indices)
    # [batch_size, question_len, num_question_neighbors, input_dim]
    passage_neighbor_node_representations = collect_neighbor_node_representations(
                                                    in_passage_repres, passage_neighbor_indices)
    # [batch_size, passage_len, num_passage_neighbors, input_dim]
                
    question_neighbor_representations = tf.concat(3, [question_neighbor_node_representations, question_neighbor_edge_representations])
    # [batch_size, question_len, num_question_neighbors, input_dim+ edge_dim]
    passage_neighbor_representations = tf.concat(3, [passage_neighbor_node_representations, passage_neighbor_edge_representations])
    # [batch_size, passage_len, num_passage_neighbors, input_dim + edge_dim]

    # =====compress neighbor_representations 
    compress_vector_dim = neighbor_vector_dim
    w_compress = tf.get_variable("w_compress", [input_dim + edge_dim, compress_vector_dim], dtype=tf.float32)
    b_compress = tf.get_variable("b_compress", [compress_vector_dim], dtype=tf.float32)

    question_neighbor_representations = tf.reshape(question_neighbor_representations, [-1, input_dim + edge_dim])
    question_neighbor_representations = tf.matmul(question_neighbor_representations, w_compress) + b_compress
    question_neighbor_representations = tf.tanh(question_neighbor_representations)
    # [batch_size*question_len*num_question_neighbors, compress_vector_dim]

    passage_neighbor_representations = tf.reshape(passage_neighbor_representations, [-1, input_dim + edge_dim])
    passage_neighbor_representations = tf.matmul(passage_neighbor_representations, w_compress) + b_compress
    passage_neighbor_representations = tf.tanh(passage_neighbor_representations)
    # [batch_size*passage_len*num_passage_neighbors, compress_vector_dim]
                
    # assume each node has a neighbor vector, and it is None at the beginning
    question_node_hidden = tf.zeros([batch_size, question_len, neighbor_vector_dim])
    question_node_cell = tf.zeros([batch_size, question_len, neighbor_vector_dim])

    passage_node_hidden = tf.zeros([batch_size, passage_len, neighbor_vector_dim])
    passage_node_cell = tf.zeros([batch_size, passage_len, neighbor_vector_dim])
        
        
    w_ingate = tf.get_variable("w_ingate", [compress_vector_dim, neighbor_vector_dim], dtype=tf.float32)
    u_ingate = tf.get_variable("u_ingate", [neighbor_vector_dim, neighbor_vector_dim], dtype=tf.float32)
    b_ingate = tf.get_variable("b_ingate", [neighbor_vector_dim], dtype=tf.float32)

    w_forgetgate = tf.get_variable("w_forgetgate", [compress_vector_dim, neighbor_vector_dim], dtype=tf.float32)
    u_forgetgate = tf.get_variable("u_forgetgate", [neighbor_vector_dim, neighbor_vector_dim], dtype=tf.float32)
    b_forgetgate = tf.get_variable("b_forgetgate", [neighbor_vector_dim], dtype=tf.float32)

    w_outgate = tf.get_variable("w_outgate", [compress_vector_dim, neighbor_vector_dim], dtype=tf.float32)
    u_outgate = tf.get_variable("u_outgate", [neighbor_vector_dim, neighbor_vector_dim], dtype=tf.float32)
    b_outgate = tf.get_variable("b_outgate", [neighbor_vector_dim], dtype=tf.float32)

    w_cell = tf.get_variable("w_cell", [compress_vector_dim, neighbor_vector_dim], dtype=tf.float32)
    u_cell = tf.get_variable("u_cell", [neighbor_vector_dim, neighbor_vector_dim], dtype=tf.float32)
    b_cell = tf.get_variable("b_cell", [neighbor_vector_dim], dtype=tf.float32)

    for i in xrange(num_syntax_match_layer):
        with tf.variable_scope('syntax_match_layer-{}'.format(i)):
            # ========for question============
            question_edge_prev_hidden = collect_neighbor_node_representations(question_node_hidden, question_neighbor_indices)
            # [batch_size, question_len, num_question_neighbors, neighbor_vector_dim]
            question_edge_prev_cell = collect_neighbor_node_representations(question_node_cell, question_neighbor_indices)
            # [batch_size, question_len, num_question_neighbors, neighbor_vector_dim]
            question_edge_prev_hidden = tf.reshape(question_edge_prev_hidden, [-1, neighbor_vector_dim])
            question_edge_prev_cell = tf.reshape(question_edge_prev_cell, [-1, neighbor_vector_dim])

            question_edge_ingate = tf.sigmoid(tf.matmul(question_neighbor_representations,w_ingate) 
                                          + tf.matmul(question_edge_prev_hidden, u_ingate) + b_ingate)
            question_edge_forgetgate = tf.sigmoid(tf.matmul(question_neighbor_representations,w_forgetgate) 
                                          + tf.matmul(question_edge_prev_hidden, u_forgetgate) + b_forgetgate)
            question_edge_outgate = tf.sigmoid(tf.matmul(question_neighbor_representations,w_outgate) 
                                          + tf.matmul(question_edge_prev_hidden, u_outgate) + b_outgate)
            question_edge_cell_input = tf.tanh(tf.matmul(question_neighbor_representations,w_cell) 
                                          + tf.matmul(question_edge_prev_hidden, u_cell) + b_cell)
            question_edge_cell = question_edge_forgetgate * question_edge_prev_cell + question_edge_ingate * question_edge_cell_input
            question_edge_hidden = question_edge_outgate * tf.tanh(question_edge_cell)
            question_edge_cell = tf.reshape(question_edge_cell, [batch_size, question_len, num_question_neighbors, neighbor_vector_dim])
            question_edge_hidden = tf.reshape(question_edge_hidden, [batch_size, question_len, num_question_neighbors, neighbor_vector_dim])
            # edge mask
            question_edge_cell = tf.mul(question_edge_cell, tf.expand_dims(question_neighbor_mask, axis=-1))
            question_edge_hidden = tf.mul(question_edge_hidden, tf.expand_dims(question_neighbor_mask, axis=-1))
            question_node_cell = tf.reduce_sum(question_edge_cell, axis=2)
            question_node_hidden = tf.reduce_sum(question_edge_hidden, axis=2)
            #[batch_size, question_len, neighbor_vector_dim]

            # node mask
            question_node_cell = question_node_cell * tf.expand_dims(question_mask, axis=-1)
            question_node_hidden = question_node_hidden * tf.expand_dims(question_mask, axis=-1)

            # ========for passage============
            passage_edge_prev_hidden = collect_neighbor_node_representations(passage_node_hidden, passage_neighbor_indices)
            passage_edge_prev_cell = collect_neighbor_node_representations(passage_node_cell, passage_neighbor_indices)
            # [batch_size, passage_len, num_passage_neighbors, neighbor_vector_dim]
            passage_edge_prev_hidden = tf.reshape(passage_edge_prev_hidden, [-1, neighbor_vector_dim])
            passage_edge_prev_cell = tf.reshape(passage_edge_prev_cell, [-1, neighbor_vector_dim])

            passage_edge_ingate = tf.sigmoid(tf.matmul(passage_neighbor_representations,w_ingate) 
                                          + tf.matmul(passage_edge_prev_hidden, u_ingate) + b_ingate)
            passage_edge_forgetgate = tf.sigmoid(tf.matmul(passage_neighbor_representations,w_forgetgate) 
                                          + tf.matmul(passage_edge_prev_hidden, u_forgetgate) + b_forgetgate)
            passage_edge_outgate = tf.sigmoid(tf.matmul(passage_neighbor_representations,w_outgate) 
                                          + tf.matmul(passage_edge_prev_hidden, u_outgate) + b_outgate)
            passage_edge_cell_input = tf.tanh(tf.matmul(passage_neighbor_representations,w_cell) 
                                          + tf.matmul(passage_edge_prev_hidden, u_cell) + b_cell)
            passage_edge_cell = passage_edge_forgetgate * passage_edge_prev_cell + passage_edge_ingate * passage_edge_cell_input
            passage_edge_hidden = passage_edge_outgate * tf.tanh(passage_edge_cell)
            passage_edge_cell = tf.reshape(passage_edge_cell, [batch_size, passage_len, num_passage_neighbors, neighbor_vector_dim])
            passage_edge_hidden = tf.reshape(passage_edge_hidden, [batch_size, passage_len, num_passage_neighbors, neighbor_vector_dim])
            # edge mask
            passage_edge_cell = tf.mul(passage_edge_cell, tf.expand_dims(passage_neighbor_mask, axis=-1))
            passage_edge_hidden = tf.mul(passage_edge_hidden, tf.expand_dims(passage_neighbor_mask, axis=-1))
            passage_node_cell = tf.reduce_sum(passage_edge_cell, axis=2)
            passage_node_hidden = tf.reduce_sum(passage_edge_hidden, axis=2)

            # node mask
            passage_node_cell = passage_node_cell * tf.expand_dims(passage_mask, axis=-1)
            passage_node_hidden = passage_node_hidden * tf.expand_dims(passage_mask, axis=-1)
                        
            #=====matching
            (node_matching_vectors, node_matching_dim) = match_passage_with_question(
                                passage_node_hidden, None, passage_mask, question_node_hidden, None,question_mask, neighbor_vector_dim, 
                                with_full_matching=False, 
                                with_attentive_matching=with_attentive_matching, 
                                with_max_attentive_matching=with_max_attentive_matching, 
                                with_maxpooling_matching=with_maxpooling_matching, 
                                with_forward_match=True, with_backward_match=False, match_options=match_options)
            all_matching_vectors.extend(node_matching_vectors) #[batch_size, passage_len, node_matching_dim]
            all_matching_dim += node_matching_dim

    all_matching_vectors = tf.concat(2, all_matching_vectors) # [batch_size, passage_len, all_matching_dim]
    return (all_matching_vectors, all_matching_dim)

def graph_matching_for_chunk_ranking(in_question_repres, in_passage_repres, 
                question_mask, passage_mask, edge_embedding,
                question_neighbor_indices, passage_neighbor_indices,
                question_neighbor_edges, passage_neighbor_edges,
                question_neighbor_mask, passage_neighbor_mask,
                question_word_idx, candidate_node_idx, candidate_mask,
                neighbor_vector_dim, input_dim, edge_dim, num_syntax_match_layer,
                with_attentive_matching=True, with_max_attentive_matching=True,
                with_maxpooling_matching=True, with_local_attentive_matching=True,match_options=None): 

    all_matching_vectors = []
    all_matching_dim = 0
    
    
    input_shape = tf.shape(question_neighbor_indices)
    batch_size = input_shape[0]
    question_len = input_shape[1]
    num_question_neighbors = input_shape[2]

    input_shape = tf.shape(passage_neighbor_indices)
#     batch_size = input_shape[0]
    passage_len = input_shape[1]
    num_passage_neighbors = input_shape[2]
    
#     question_neighbor_mask = tf.sequence_mask(tf.reshape(question_neighbor_size, [-1]), num_question_neighbors, dtype=tf.float32)
#     question_neighbor_mask = tf.reshape(question_neighbor_mask, [batch_size, question_len, num_question_neighbors])

#     passage_neighbor_mask = tf.sequence_mask(tf.reshape(passage_neighbor_size, [-1]), num_passage_neighbors, dtype=tf.float32)
#     passage_neighbor_mask = tf.reshape(passage_neighbor_mask, [batch_size, passage_len, num_passage_neighbors])

    question_neighbor_edge_representations = tf.nn.embedding_lookup(edge_embedding, question_neighbor_edges) 
    # [batch_size, question_len, num_question_neighbors, edge_dim]
    passage_neighbor_edge_representations = tf.nn.embedding_lookup(edge_embedding, passage_neighbor_edges) 
    # [batch_size, passage_len, num_passage_neighbors, edge_dim]
    question_neighbor_node_representations = collect_neighbor_node_representations(
                                                    in_question_repres, question_neighbor_indices)
    # [batch_size, question_len, num_question_neighbors, input_dim]
    passage_neighbor_node_representations = collect_neighbor_node_representations(
                                                    in_passage_repres, passage_neighbor_indices)
    # [batch_size, passage_len, num_passage_neighbors, input_dim]
                
    question_neighbor_representations = tf.concat(3, [question_neighbor_node_representations, question_neighbor_edge_representations])
    # [batch_size, question_len, num_question_neighbors, input_dim+ edge_dim]
    passage_neighbor_representations = tf.concat(3, [passage_neighbor_node_representations, passage_neighbor_edge_representations])
    # [batch_size, passage_len, num_passage_neighbors, input_dim + edge_dim]

    # =====compress neighbor_representations 
    compress_vector_dim = neighbor_vector_dim
    w_compress = tf.get_variable("w_compress", [input_dim + edge_dim, compress_vector_dim], dtype=tf.float32)
    b_compress = tf.get_variable("b_compress", [compress_vector_dim], dtype=tf.float32)

    question_neighbor_representations = tf.reshape(question_neighbor_representations, [-1, input_dim + edge_dim])
    question_neighbor_representations = tf.matmul(question_neighbor_representations, w_compress) + b_compress
    question_neighbor_representations = tf.tanh(question_neighbor_representations)
    # [batch_size*question_len*num_question_neighbors, compress_vector_dim]

    passage_neighbor_representations = tf.reshape(passage_neighbor_representations, [-1, input_dim + edge_dim])
    passage_neighbor_representations = tf.matmul(passage_neighbor_representations, w_compress) + b_compress
    passage_neighbor_representations = tf.tanh(passage_neighbor_representations)
    # [batch_size*passage_len*num_passage_neighbors, compress_vector_dim]
                
    # assume each node has a neighbor vector, and it is None at the beginning
    question_node_hidden = tf.zeros([batch_size, question_len, neighbor_vector_dim])
    question_node_cell = tf.zeros([batch_size, question_len, neighbor_vector_dim])

    passage_node_hidden = tf.zeros([batch_size, passage_len, neighbor_vector_dim])
    passage_node_cell = tf.zeros([batch_size, passage_len, neighbor_vector_dim])
        
        
    w_ingate = tf.get_variable("w_ingate", [compress_vector_dim, neighbor_vector_dim], dtype=tf.float32)
    u_ingate = tf.get_variable("u_ingate", [neighbor_vector_dim, neighbor_vector_dim], dtype=tf.float32)
    b_ingate = tf.get_variable("b_ingate", [neighbor_vector_dim], dtype=tf.float32)

    w_forgetgate = tf.get_variable("w_forgetgate", [compress_vector_dim, neighbor_vector_dim], dtype=tf.float32)
    u_forgetgate = tf.get_variable("u_forgetgate", [neighbor_vector_dim, neighbor_vector_dim], dtype=tf.float32)
    b_forgetgate = tf.get_variable("b_forgetgate", [neighbor_vector_dim], dtype=tf.float32)

    w_outgate = tf.get_variable("w_outgate", [compress_vector_dim, neighbor_vector_dim], dtype=tf.float32)
    u_outgate = tf.get_variable("u_outgate", [neighbor_vector_dim, neighbor_vector_dim], dtype=tf.float32)
    b_outgate = tf.get_variable("b_outgate", [neighbor_vector_dim], dtype=tf.float32)

    w_cell = tf.get_variable("w_cell", [compress_vector_dim, neighbor_vector_dim], dtype=tf.float32)
    u_cell = tf.get_variable("u_cell", [neighbor_vector_dim, neighbor_vector_dim], dtype=tf.float32)
    b_cell = tf.get_variable("b_cell", [neighbor_vector_dim], dtype=tf.float32)
    
    # calculate question graph representation
    all_question_graph_representations = []
    for i in xrange(num_syntax_match_layer):
        with tf.variable_scope('syntax_match_layer-{}'.format(i)):
            # ========for question============
            question_edge_prev_hidden = collect_neighbor_node_representations(question_node_hidden, question_neighbor_indices)
            # [batch_size, question_len, num_question_neighbors, neighbor_vector_dim]
            question_edge_prev_cell = collect_neighbor_node_representations(question_node_cell, question_neighbor_indices)
            # [batch_size, question_len, num_question_neighbors, neighbor_vector_dim]
            question_edge_prev_hidden = tf.reshape(question_edge_prev_hidden, [-1, neighbor_vector_dim])
            question_edge_prev_cell = tf.reshape(question_edge_prev_cell, [-1, neighbor_vector_dim])

            question_edge_ingate = tf.sigmoid(tf.matmul(question_neighbor_representations,w_ingate) 
                                          + tf.matmul(question_edge_prev_hidden, u_ingate) + b_ingate)
            question_edge_forgetgate = tf.sigmoid(tf.matmul(question_neighbor_representations,w_forgetgate) 
                                          + tf.matmul(question_edge_prev_hidden, u_forgetgate) + b_forgetgate)
            question_edge_outgate = tf.sigmoid(tf.matmul(question_neighbor_representations,w_outgate) 
                                          + tf.matmul(question_edge_prev_hidden, u_outgate) + b_outgate)
            question_edge_cell_input = tf.tanh(tf.matmul(question_neighbor_representations,w_cell) 
                                          + tf.matmul(question_edge_prev_hidden, u_cell) + b_cell)
            question_edge_cell = question_edge_forgetgate * question_edge_prev_cell + question_edge_ingate * question_edge_cell_input
            question_edge_hidden = question_edge_outgate * tf.tanh(question_edge_cell)
            question_edge_cell = tf.reshape(question_edge_cell, [batch_size, question_len, num_question_neighbors, neighbor_vector_dim])
            question_edge_hidden = tf.reshape(question_edge_hidden, [batch_size, question_len, num_question_neighbors, neighbor_vector_dim])
            # edge mask
            question_edge_cell = tf.mul(question_edge_cell, tf.expand_dims(question_neighbor_mask, axis=-1))
            question_edge_hidden = tf.mul(question_edge_hidden, tf.expand_dims(question_neighbor_mask, axis=-1))
#             question_node_cell = tf.reduce_sum(question_edge_cell, axis=2)
#             question_node_hidden = tf.reduce_sum(question_edge_hidden, axis=2)
            question_node_cell = tf.reduce_max(question_edge_cell, axis=2)
            question_node_hidden = tf.reduce_max(question_edge_hidden, axis=2) # TODO
            #[batch_size, question_len, neighbor_vector_dim]

            # node mask
            question_node_cell = question_node_cell * tf.expand_dims(question_mask, axis=-1)
            question_node_hidden = question_node_hidden * tf.expand_dims(question_mask, axis=-1)

            question_word_representation = collect_node_representations(question_node_hidden, question_word_idx)
            # [batch_size, neighbor_vector_dim]
            all_question_graph_representations.append(tf.reshape(question_word_representation, [batch_size, 1, neighbor_vector_dim]))
    all_question_graph_representations = tf.concat(1, all_question_graph_representations) # [batch_size, num_match_layer, neighbor_vector_dim]
     
    # calculate passage representation and match it with the question
    for i in xrange(num_syntax_match_layer):
        with tf.variable_scope('syntax_match_layer-{}'.format(i)):
            passage_edge_prev_hidden = collect_neighbor_node_representations(passage_node_hidden, passage_neighbor_indices)
            passage_edge_prev_cell = collect_neighbor_node_representations(passage_node_cell, passage_neighbor_indices)
            # [batch_size, passage_len, num_passage_neighbors, neighbor_vector_dim]
            passage_edge_prev_hidden = tf.reshape(passage_edge_prev_hidden, [-1, neighbor_vector_dim])
            passage_edge_prev_cell = tf.reshape(passage_edge_prev_cell, [-1, neighbor_vector_dim])

            passage_edge_ingate = tf.sigmoid(tf.matmul(passage_neighbor_representations,w_ingate) 
                                          + tf.matmul(passage_edge_prev_hidden, u_ingate) + b_ingate)
            passage_edge_forgetgate = tf.sigmoid(tf.matmul(passage_neighbor_representations,w_forgetgate) 
                                          + tf.matmul(passage_edge_prev_hidden, u_forgetgate) + b_forgetgate)
            passage_edge_outgate = tf.sigmoid(tf.matmul(passage_neighbor_representations,w_outgate) 
                                          + tf.matmul(passage_edge_prev_hidden, u_outgate) + b_outgate)
            passage_edge_cell_input = tf.tanh(tf.matmul(passage_neighbor_representations,w_cell) 
                                          + tf.matmul(passage_edge_prev_hidden, u_cell) + b_cell)
            passage_edge_cell = passage_edge_forgetgate * passage_edge_prev_cell + passage_edge_ingate * passage_edge_cell_input
            passage_edge_hidden = passage_edge_outgate * tf.tanh(passage_edge_cell)
            passage_edge_cell = tf.reshape(passage_edge_cell, [batch_size, passage_len, num_passage_neighbors, neighbor_vector_dim])
            passage_edge_hidden = tf.reshape(passage_edge_hidden, [batch_size, passage_len, num_passage_neighbors, neighbor_vector_dim])
            # edge mask
            passage_edge_cell = tf.mul(passage_edge_cell, tf.expand_dims(passage_neighbor_mask, axis=-1))
            passage_edge_hidden = tf.mul(passage_edge_hidden, tf.expand_dims(passage_neighbor_mask, axis=-1))
#             passage_node_cell = tf.reduce_sum(passage_edge_cell, axis=2)
#             passage_node_hidden = tf.reduce_sum(passage_edge_hidden, axis=2)
            passage_node_cell = tf.reduce_max(passage_edge_cell, axis=2)
            passage_node_hidden = tf.reduce_max(passage_edge_hidden, axis=2) # TODO

            # node mask
            passage_node_cell = passage_node_cell * tf.expand_dims(passage_mask, axis=-1)
            passage_node_hidden = passage_node_hidden * tf.expand_dims(passage_mask, axis=-1)
                        
            #=====matching
            canidate_node_representation = collect_node_representations(passage_node_hidden, candidate_node_idx)
            # [batch_size, num_candidate_nodes, neighbor_vector_dim]
            (node_matching_vectors, node_matching_dim) = match_passage_with_question(
                                canidate_node_representation, None, candidate_mask, 
                                all_question_graph_representations, None,None, 
                                neighbor_vector_dim, with_full_matching=False, 
                                with_attentive_matching=with_attentive_matching, 
                                with_max_attentive_matching=with_max_attentive_matching, 
                                with_maxpooling_matching=with_maxpooling_matching, 
                                with_local_attentive_matching=with_local_attentive_matching,
                                with_forward_match=True, with_backward_match=False, match_options=match_options)
            all_matching_vectors.extend(node_matching_vectors) #[batch_size, num_candidate_nodes, node_matching_dim]
            all_matching_dim += node_matching_dim

    all_matching_vectors = tf.concat(2, all_matching_vectors) # [batch_size, num_candidate_nodes, all_matching_dim]
    return (all_matching_vectors, all_matching_dim)

In [0]:
def collect_neighbor_node_representations(representation, positions):
    # representation: [batch_size, num_nodes, feature_dim]
    # positions: [batch_size, num_nodes, num_neighbors]
    feature_dim = tf.shape(representation)[2]
    input_shape = tf.shape(positions)
    batch_size = input_shape[0]
    num_nodes = input_shape[1]
    num_neighbors = input_shape[2]
    positions_flat = tf.reshape(positions, [batch_size, num_nodes*num_neighbors])
    def singel_instance(x):
        # x[0]: [num_nodes, feature_dim]
        # x[1]: [num_nodes*num_neighbors]
        return tf.gather(x[0], x[1])
    elems = (representation, positions_flat)
    representations = tf.map_fn(singel_instance, elems, dtype=tf.float32)
    return tf.reshape(representations, [batch_size, num_nodes, num_neighbors, feature_dim])

def collect_final_step_lstm(lstm_rep, lens):
    lens = tf.maximum(lens, tf.zeros_like(lens, dtype=tf.int32)) # [batch,]
    idxs = tf.range(0, limit=tf.shape(lens)[0]) # [batch,]
    indices = tf.stack((idxs,lens,), axis=1) # [batch_size, 2]
    return tf.gather_nd(lstm_rep, indices, name='lstm-forward-last')

class GraphEncoder(object):
    def __init__(self, word_vocab=None, edge_label_vocab=None, char_vocab=None, is_training=True, options=None):
        assert options != None

        self.passage_nodes_size = tf.placeholder(tf.int32, [None]) # [batch_size]
        self.passage_nodes = tf.placeholder(tf.int32, [None, None]) # [batch_size, passage_nodes_size_max]
        if options.with_char:
            self.passage_nodes_chars_size = tf.placeholder(tf.int32, [None, None])
            self.passage_nodes_chars = tf.placeholder(tf.int32, [None, None, None])

        # [batch_size, passage_nodes_size_max, passage_neighbors_size_max]
        self.passage_in_neighbor_indices = tf.placeholder(tf.int32, [None, None, None])
        self.passage_in_neighbor_edges = tf.placeholder(tf.int32, [None, None, None])
        self.passage_in_neighbor_mask = tf.placeholder(tf.float32, [None, None, None])

        # [batch_size, passage_nodes_size_max, passage_neighbors_size_max]
        self.passage_out_neighbor_indices = tf.placeholder(tf.int32, [None, None, None])
        self.passage_out_neighbor_edges = tf.placeholder(tf.int32, [None, None, None])
        self.passage_out_neighbor_mask = tf.placeholder(tf.float32, [None, None, None])

        # shapes
        input_shape = tf.shape(self.passage_in_neighbor_indices)
        batch_size = input_shape[0]
        passage_nodes_size_max = input_shape[1]
        passage_in_neighbors_size_max = input_shape[2]
        passage_out_neighbors_size_max = tf.shape(self.passage_out_neighbor_indices)[2]
        if options.with_char:
            passage_nodes_chars_size_max = tf.shape(self.passage_nodes_chars)[2]

        # masks
        # [batch_size, passage_nodes_size_max]
        self.passage_nodes_mask = tf.sequence_mask(self.passage_nodes_size, passage_nodes_size_max, dtype=tf.float32)

        # embeddings
        if options.fix_word_vec:
            word_vec_trainable = False
            cur_device = '/cpu:0'
        else:
            word_vec_trainable = True
            cur_device = '/gpu:0'
        with tf.device(cur_device):
            self.word_embedding = tf.get_variable("word_embedding", trainable=word_vec_trainable,
                                                  initializer=tf.constant(word_vocab.word_vecs), dtype=tf.float32)

        self.edge_embedding = tf.get_variable("edge_embedding",
                initializer=tf.constant(edge_label_vocab.word_vecs), dtype=tf.float32)

        word_dim = word_vocab.word_dim
        edge_dim = edge_label_vocab.word_dim

        if options.with_char:
            self.char_embedding = tf.get_variable("char_embedding",
                    initializer=tf.constant(char_vocab.word_vecs), dtype=tf.float32)
            char_dim = char_vocab.word_dim

        # word representation for nodes, where each node only includes one word
        # [batch_size, passage_nodes_size_max, word_dim]
        passage_node_representation = tf.nn.embedding_lookup(self.word_embedding, self.passage_nodes)

        if options.with_char:
            # [batch_size, passage_nodes_size_max, passage_nodes_chars_size_max, char_dim]
            passage_nodes_chars_representation = tf.nn.embedding_lookup(self.char_embedding, self.passage_nodes_chars)
            passage_nodes_chars_representation = tf.reshape(passage_nodes_chars_representation,
                    shape=[batch_size*passage_nodes_size_max, passage_nodes_chars_size_max, char_dim])
            passage_nodes_chars_size = tf.reshape(self.passage_nodes_chars_size, [batch_size*passage_nodes_size_max])
            with tf.variable_scope('node_char_lstm'):
                node_char_lstm_cell = tf.contrib.rnn.LSTMCell(options.char_lstm_dim)
                node_char_lstm_cell = tf.contrib.rnn.MultiRNNCell([node_char_lstm_cell])
                # [batch_size*node_num, char_num, char_lstm_dim]
                node_char_outputs = tf.nn.dynamic_rnn(node_char_lstm_cell, passage_nodes_chars_representation,
                        sequence_length=passage_nodes_chars_size, dtype=tf.float32)[0]
                node_char_outputs = collect_final_step_lstm(node_char_outputs, passage_nodes_chars_size-1)
                # [batch_size, node_num, char_lstm_dim]
                node_char_outputs = tf.reshape(node_char_outputs, [batch_size, passage_nodes_size_max, options.char_lstm_dim])

        if options.with_char:
            input_dim = word_dim + options.char_lstm_dim
            passage_node_representation = tf.concat([passage_node_representation, node_char_outputs], 2)
        else:
            input_dim = word_dim
            passage_node_representation = passage_node_representation

        # apply the mask
        passage_node_representation = passage_node_representation * tf.expand_dims(self.passage_nodes_mask, axis=-1)

        if options.compress_input: # compress input word vector into smaller vectors
            w_compress = tf.get_variable("w_compress_input", [input_dim, options.compress_input_dim], dtype=tf.float32)
            b_compress = tf.get_variable("b_compress_input", [options.compress_input_dim], dtype=tf.float32)

            passage_node_representation = tf.reshape(passage_node_representation, [-1, input_dim])
            passage_node_representation = tf.matmul(passage_node_representation, w_compress) + b_compress
            passage_node_representation = tf.tanh(passage_node_representation)
            passage_node_representation = tf.reshape(passage_node_representation, \
                    [batch_size, passage_nodes_size_max, options.compress_input_dim])
            input_dim = options.compress_input_dim

        if is_training:
            passage_node_representation = tf.nn.dropout(passage_node_representation, (1 - options.dropout_rate))

        # ======Highway layer======
        if options.with_highway:
            with tf.variable_scope("input_highway"):
                passage_node_representation = multi_highway_layer(passage_node_representation,
                        input_dim, options.highway_layer_num)

        self.input_dim = input_dim

        with tf.variable_scope('graph_encoder'):
            # =========== in neighbor
            # [batch_size, passage_len, passage_neighbors_size_max, edge_dim]
            passage_in_neighbor_edge_representations = tf.nn.embedding_lookup(self.edge_embedding,
                    self.passage_in_neighbor_edges)
            # [batch_size, passage_len, passage_neighbors_size_max, node_dim]
            passage_in_neighbor_node_representations = collect_neighbor_node_representations(
                    passage_node_representation, self.passage_in_neighbor_indices)

            passage_in_neighbor_representations = tf.concat( \
                    [passage_in_neighbor_node_representations, passage_in_neighbor_edge_representations], 3)
            passage_in_neighbor_representations = tf.multiply(passage_in_neighbor_representations,
                    tf.expand_dims(self.passage_in_neighbor_mask, axis=-1))
            # [batch_size, passage_len, node_dim + edge_dim]
            passage_in_neighbor_representations = tf.reduce_sum(passage_in_neighbor_representations, axis=2)

            # ============ out neighbor
            # [batch_size, passage_len, passage_neighbors_size_max, edge_dim]
            passage_out_neighbor_edge_representations = tf.nn.embedding_lookup(self.edge_embedding,
                    self.passage_out_neighbor_edges)
            # [batch_size, passage_len, passage_neighbors_size_max, node_dim]
            passage_out_neighbor_node_representations = collect_neighbor_node_representations(
                    passage_node_representation, self.passage_out_neighbor_indices)

            passage_out_neighbor_representations = tf.concat( \
                    [passage_out_neighbor_node_representations, passage_out_neighbor_edge_representations], 3)
            passage_out_neighbor_representations = tf.multiply(passage_out_neighbor_representations,
                    tf.expand_dims(self.passage_out_neighbor_mask, axis=-1))
            # [batch_size, passage_len, node_dim + edge_dim]
            passage_out_neighbor_representations = tf.reduce_sum(passage_out_neighbor_representations, axis=2)

            # =====transpose neighbor_representations
            grn_hidden_dim = options.neighbor_vector_dim
            w_trans = tf.get_variable("w_trans", [input_dim + edge_dim, grn_hidden_dim], dtype=tf.float32)
            b_trans = tf.get_variable("b_trans", [grn_hidden_dim], dtype=tf.float32)

            passage_in_neighbor_representations = tf.reshape(passage_in_neighbor_representations,
                    [-1, input_dim + edge_dim])
            passage_in_neighbor_representations = tf.matmul(passage_in_neighbor_representations, w_trans) + b_trans
            passage_in_neighbor_representations = tf.tanh(passage_in_neighbor_representations)

            passage_out_neighbor_representations = tf.reshape(passage_out_neighbor_representations,
                    [-1, input_dim + edge_dim])
            passage_out_neighbor_representations = tf.matmul(passage_out_neighbor_representations, w_trans) + b_trans
            passage_out_neighbor_representations = tf.tanh(passage_out_neighbor_representations)

            # assume each node has a neighbor vector, and it is None at the beginning
            passage_node_hidden = tf.zeros([batch_size, passage_nodes_size_max, grn_hidden_dim])
            passage_node_cell = tf.zeros([batch_size, passage_nodes_size_max, grn_hidden_dim])

            w_in_ingate = tf.get_variable("w_in_ingate",
                    [grn_hidden_dim, grn_hidden_dim], dtype=tf.float32)
            u_in_ingate = tf.get_variable("u_in_ingate",
                    [grn_hidden_dim, grn_hidden_dim], dtype=tf.float32)
            b_ingate = tf.get_variable("b_in_ingate",
                    [grn_hidden_dim], dtype=tf.float32)
            w_out_ingate = tf.get_variable("w_out_ingate",
                    [grn_hidden_dim, grn_hidden_dim], dtype=tf.float32)
            u_out_ingate = tf.get_variable("u_out_ingate",
                    [grn_hidden_dim, grn_hidden_dim], dtype=tf.float32)

            w_in_forgetgate = tf.get_variable("w_in_forgetgate",
                    [grn_hidden_dim, grn_hidden_dim], dtype=tf.float32)
            u_in_forgetgate = tf.get_variable("u_in_forgetgate",
                    [grn_hidden_dim, grn_hidden_dim], dtype=tf.float32)
            b_forgetgate = tf.get_variable("b_in_forgetgate",
                    [grn_hidden_dim], dtype=tf.float32)
            w_out_forgetgate = tf.get_variable("w_out_forgetgate",
                    [grn_hidden_dim, grn_hidden_dim], dtype=tf.float32)
            u_out_forgetgate = tf.get_variable("u_out_forgetgate",
                    [grn_hidden_dim, grn_hidden_dim], dtype=tf.float32)

            w_in_outgate = tf.get_variable("w_in_outgate",
                    [grn_hidden_dim, grn_hidden_dim], dtype=tf.float32)
            u_in_outgate = tf.get_variable("u_in_outgate",
                    [grn_hidden_dim, grn_hidden_dim], dtype=tf.float32)
            b_outgate = tf.get_variable("b_in_outgate",
                    [grn_hidden_dim], dtype=tf.float32)
            w_out_outgate = tf.get_variable("w_out_outgate",
                    [grn_hidden_dim, grn_hidden_dim], dtype=tf.float32)
            u_out_outgate = tf.get_variable("u_out_outgate",
                    [grn_hidden_dim, grn_hidden_dim], dtype=tf.float32)

            w_in_cell = tf.get_variable("w_in_cell",
                    [grn_hidden_dim, grn_hidden_dim], dtype=tf.float32)
            u_in_cell = tf.get_variable("u_in_cell",
                    [grn_hidden_dim, grn_hidden_dim], dtype=tf.float32)
            b_cell = tf.get_variable("b_in_cell",
                    [grn_hidden_dim], dtype=tf.float32)
            w_out_cell = tf.get_variable("w_out_cell",
                    [grn_hidden_dim, grn_hidden_dim], dtype=tf.float32)
            u_out_cell = tf.get_variable("u_out_cell",
                    [grn_hidden_dim, grn_hidden_dim], dtype=tf.float32)

            # calculate question graph representation
            graph_representations = []
            for i in xrange(options.num_syntax_match_layer):
                # =============== in edge hidden
                # h_{ij} [batch_size, node_len, neighbors_size, neighbor_vector_dim]
                passage_in_edge_prev_hidden = collect_neighbor_node_representations(passage_node_hidden,
                        self.passage_in_neighbor_indices)
                passage_in_edge_prev_hidden = tf.multiply(passage_in_edge_prev_hidden,
                    tf.expand_dims(self.passage_in_neighbor_mask, axis=-1))
                # [batch_size, node_len, neighbor_vector_dim]
                passage_in_edge_prev_hidden = tf.reduce_sum(passage_in_edge_prev_hidden, axis=2)
                passage_in_edge_prev_hidden = tf.multiply(passage_in_edge_prev_hidden,
                        tf.expand_dims(self.passage_nodes_mask, axis=-1))
                passage_in_edge_prev_hidden = tf.reshape(passage_in_edge_prev_hidden,
                        [-1, grn_hidden_dim])

                # =============== out edge hidden
                # h_{jk} [batch_size, node_len, neighbors_size, neighbor_vector_dim]
                passage_out_edge_prev_hidden = collect_neighbor_node_representations(passage_node_hidden,
                        self.passage_out_neighbor_indices)
                passage_out_edge_prev_hidden = tf.multiply(passage_out_edge_prev_hidden,
                    tf.expand_dims(self.passage_out_neighbor_mask, axis=-1))
                # [batch_size, node_len, neighbor_vector_dim]
                passage_out_edge_prev_hidden = tf.reduce_sum(passage_out_edge_prev_hidden, axis=2)
                passage_out_edge_prev_hidden = tf.multiply(passage_out_edge_prev_hidden,
                        tf.expand_dims(self.passage_nodes_mask, axis=-1))
                passage_out_edge_prev_hidden = tf.reshape(passage_out_edge_prev_hidden,
                        [-1, grn_hidden_dim])

                ## ig
                passage_edge_ingate = tf.sigmoid(tf.matmul(passage_in_neighbor_representations, w_in_ingate)
                                          + tf.matmul(passage_in_edge_prev_hidden, u_in_ingate)
                                          + tf.matmul(passage_out_neighbor_representations, w_out_ingate)
                                          + tf.matmul(passage_out_edge_prev_hidden, u_out_ingate)
                                          + b_ingate)
                passage_edge_ingate = tf.reshape(passage_edge_ingate,
                        [batch_size, passage_nodes_size_max, grn_hidden_dim])
                ## fg
                passage_edge_forgetgate = tf.sigmoid(tf.matmul(passage_in_neighbor_representations, w_in_forgetgate)
                                          + tf.matmul(passage_in_edge_prev_hidden, u_in_forgetgate)
                                          + tf.matmul(passage_out_neighbor_representations, w_out_forgetgate)
                                          + tf.matmul(passage_out_edge_prev_hidden, u_out_forgetgate)
                                          + b_forgetgate)
                passage_edge_forgetgate = tf.reshape(passage_edge_forgetgate,
                        [batch_size, passage_nodes_size_max, grn_hidden_dim])
                ## og
                passage_edge_outgate = tf.sigmoid(tf.matmul(passage_in_neighbor_representations, w_in_outgate)
                                          + tf.matmul(passage_in_edge_prev_hidden, u_in_outgate)
                                          + tf.matmul(passage_out_neighbor_representations, w_out_outgate)
                                          + tf.matmul(passage_out_edge_prev_hidden, u_out_outgate)
                                          + b_outgate)
                passage_edge_outgate = tf.reshape(passage_edge_outgate,
                        [batch_size, passage_nodes_size_max, grn_hidden_dim])
                ## input
                passage_edge_cell_input = tf.tanh(tf.matmul(passage_in_neighbor_representations, w_in_cell)
                                          + tf.matmul(passage_in_edge_prev_hidden, u_in_cell)
                                          + tf.matmul(passage_out_neighbor_representations, w_out_cell)
                                          + tf.matmul(passage_out_edge_prev_hidden, u_out_cell)
                                          + b_cell)
                passage_edge_cell_input = tf.reshape(passage_edge_cell_input,
                        [batch_size, passage_nodes_size_max, grn_hidden_dim])

                passage_edge_cell = passage_edge_forgetgate * passage_node_cell + passage_edge_ingate * passage_edge_cell_input
                passage_edge_hidden = passage_edge_outgate * tf.tanh(passage_edge_cell)
                # node mask
                # [batch_size, passage_len, neighbor_vector_dim]
                passage_node_cell = tf.multiply(passage_edge_cell, tf.expand_dims(self.passage_nodes_mask, axis=-1))
                passage_node_hidden = tf.multiply(passage_edge_hidden, tf.expand_dims(self.passage_nodes_mask, axis=-1))

                graph_representations.append(passage_node_hidden)

            # decide how to use graph_representations
            self.graph_representations = graph_representations
            self.node_representations = passage_node_representation
            self.graph_hiddens = passage_node_hidden
            self.graph_cells = passage_node_cell

            self.batch_size = batch_size

In [0]:
cc = SmoothingFunction()
ROUGE_path = '/usr/local/lib/python2.7/dist-packages/pythonrouge/RELEASE-1.5.5/ROUGE-1.5.5.pl'
data_path = '/usr/local/lib/python2.7/dist-packages/pythonrouge/RELEASE-1.5.5/data'

def _clip_and_normalize(word_probs, epsilon):
    '''
    word_probs: 1D tensor of [vsize]
    '''
    word_probs = tf.clip_by_value(word_probs, epsilon, 1.0 - epsilon)
    return word_probs / tf.reduce_sum(word_probs, axis=-1, keep_dims=True) # scale preds so that the class probas of each sample sum to 1

def collect_by_indices(memory, indices): # [batch, node_num, dim], [batch, 3, 5]
    batch_size = tf.shape(indices)[0]
    entity_num = tf.shape(indices)[1]
    entity_size = tf.shape(indices)[2]
    idxs = tf.range(0, limit=batch_size) # [batch]
    idxs = tf.reshape(idxs, [-1, 1, 1]) # [batch, 1, 1]
    idxs = tf.tile(idxs, [1, entity_num, entity_size])
    indices = tf.maximum(indices, tf.zeros_like(indices, dtype=tf.int32))
    indices = tf.stack((idxs,indices), axis=3) # [batch,3,5,2]
    return tf.gather_nd(memory, indices)

def sentence_rouge(reflex, genlex):
    #rouge = Pythonrouge(n_gram=2, ROUGE_SU4=True, ROUGE_L=True, stemming=True, stopwords=True, word_level=True, length_limit=True, \
     #       length=50, use_cf=False, cf=95, scoring_formula="average", resampling=True, samples=1000, favor=True, p=0.5)
    genlex = [[genlex,]]
    reflex = [[[reflex]]]
    #setting_file = rouge.setting(files=False, summary=genlex, reference=reflex)
    #result = rouge.eval_rouge(setting_file, recall_only=False, ROUGE_path=ROUGE_path, data_path=data_path)
    rouge = Pythonrouge(summary_file_exist=False,
                    summary=genlex, reference=reflex,
                    n_gram=2, ROUGE_SU4=True, ROUGE_L=False,
                    recall_only=True, stemming=True, stopwords=True,
                    word_level=True, length_limit=True, length=50,
                    use_cf=False, cf=95, scoring_formula='average',
                    resampling=True, samples=1000, favor=True, p=0.5)
    #rouge.ROUGE_path = ROUGE_path
    #rouge.data_path = data_path
    result = rouge.calc_score()
    print(result)
    return result['ROUGE-SU4']

class ModelGraph(object):
    def __init__(self, word_vocab, char_vocab, Edgelabel_vocab, options=None, mode='train'):
        # the value of 'mode' can be:
        #  'train',
        #  'evaluate'
        self.mode = mode

        # is_training controls whether to use dropout
        is_training = True if mode in ('train', ) else False

        self.options = options
        self.word_vocab = word_vocab

        # encode the input instance
        # encoder.graph_hidden [batch, node_num, vsize]
        # encoder.graph_cell [batch, node_num, vsize]
        self.encoder = GraphEncoder(
                word_vocab = word_vocab,
                edge_label_vocab = Edgelabel_vocab,
                char_vocab = char_vocab,
                is_training = is_training, options = options)

        # ============== Choices of attention memory ================
        if options.attention_type == 'hidden':
            self.encoder_dim = options.neighbor_vector_dim
            self.encoder_states = self.encoder.graph_hiddens
        elif options.attention_type == 'hidden_cell':
            self.encoder_dim = options.neighbor_vector_dim * 2
            self.encoder_states = tf.concat([self.encoder.graph_hiddens, self.encoder.graph_cells], 2)
        elif options.attention_type == 'hidden_embed':
            self.encoder_dim = options.neighbor_vector_dim + self.encoder.input_dim
            self.encoder_states = tf.concat([self.encoder.graph_hiddens, self.encoder.node_representations], 2)
        else:
            assert False, '%s not supported yet' % options.attention_type

        self.nodes = self.encoder.passage_nodes
        self.nodes_num = self.encoder.passage_nodes_size
        if options.with_char:
            self.nodes_chars = self.encoder.passage_nodes_chars
            self.nodes_chars_num = self.encoder.passage_nodes_chars_size
        self.nodes_mask = self.encoder.passage_nodes_mask

        self.in_neigh_indices = self.encoder.passage_in_neighbor_indices
        self.in_neigh_edges = self.encoder.passage_in_neighbor_edges
        self.in_neigh_mask = self.encoder.passage_in_neighbor_mask

        self.out_neigh_indices = self.encoder.passage_out_neighbor_indices
        self.out_neigh_edges = self.encoder.passage_out_neighbor_edges
        self.out_neigh_mask = self.encoder.passage_out_neighbor_mask

        ## generating prediction results
        self.entity_indices = tf.placeholder(tf.int32, [None, None, None],
                name="entity_indices")
        self.entity_indices_mask = tf.placeholder(tf.float32, [None, None, None],
                name="entity_indices_mask")
        batch_size = tf.shape(self.encoder_states)[0]
        node_num = tf.shape(self.encoder_states)[1]
        dim = tf.shape(self.encoder_states)[2]
        entity_num = tf.shape(self.entity_indices)[1]
        entity_size = tf.shape(self.entity_indices)[2]

        # self.encoder_states [batch, node_num, encoder_dim]
        # entity_states [batch, 3, 5, dim]
        entity_states = collect_by_indices(self.encoder_states, self.entity_indices)
        # applying mask
        entity_states = entity_states * tf.expand_dims(self.entity_indices_mask, axis=-1)
        # average within each entity: [batch, 3, encoder_dim]
        entity_states = tf.reduce_mean(entity_states, axis=2)
        # flatten: [batch, 3*encoder_dim]
        entity_states = tf.reshape(entity_states, [batch_size, entity_num*dim])

        w_linear = tf.get_variable("w_linear",
                [options.entity_num*self.encoder_dim, options.class_num], dtype=tf.float32)
        b_linear = tf.get_variable("b_linear",
                [options.class_num], dtype=tf.float32)
        # [batch, class_num]
        prediction = tf.nn.softmax(tf.matmul(entity_states, w_linear) + b_linear)
        prediction = _clip_and_normalize(prediction, 1.0e-6)
        self.output = tf.argmax(prediction,axis=-1,output_type=tf.int32)

        ## calculating accuracy
        self.refs = tf.placeholder(tf.int32, [None,])
        self.accu = tf.reduce_sum(tf.cast(tf.equal(self.output,self.refs),dtype=tf.float32))

        ## calculating loss
        # xent: [batch]
        xent = -tf.reduce_sum(
                tf.one_hot(self.refs,options.class_num)*tf.log(prediction),
                axis=-1)
        self.loss = tf.reduce_mean(xent)

        if mode != 'train':
            print('Return from here, just evaluate')
            return

        if options.optimize_type == 'adadelta':
            clipper = 50
            optimizer = tf.train.AdadeltaOptimizer(learning_rate=options.learning_rate)
            tvars = tf.trainable_variables()
            if options.lambda_l2>0.0:
                l2_loss = tf.add_n([tf.nn.l2_loss(v) for v in tvars if v.get_shape().ndims > 1])
                self.loss = self.loss + options.lambda_l2 * l2_loss
            grads, _ = tf.clip_by_global_norm(tf.gradients(self.loss, tvars), clipper)
            self.train_op = optimizer.apply_gradients(zip(grads, tvars))
        elif options.optimize_type == 'adam':
            clipper = 50
            optimizer = tf.train.AdamOptimizer(learning_rate=options.learning_rate)
            tvars = tf.trainable_variables()
            if options.lambda_l2>0.0:
                l2_loss = tf.add_n([tf.nn.l2_loss(v) for v in tvars if v.get_shape().ndims > 1])
                self.loss = self.loss + options.lambda_l2 * l2_loss
            grads, _ = tf.clip_by_global_norm(tf.gradients(self.loss, tvars), clipper)
            self.train_op = optimizer.apply_gradients(zip(grads, tvars))

        extra_train_ops = []
        train_ops = [self.train_op] + extra_train_ops
        self.train_op = tf.group(*train_ops)


    def execute(self, sess, batch, options, is_train=False):
        feed_dict = {}
        feed_dict[self.nodes] = batch.nodes
        feed_dict[self.nodes_num] = batch.node_num
        if options.with_char:
            feed_dict[self.nodes_chars] = batch.nodes_chars
            feed_dict[self.nodes_chars_num] = batch.nodes_chars_num

        feed_dict[self.in_neigh_indices] = batch.in_neigh_indices
        feed_dict[self.in_neigh_edges] = batch.in_neigh_edges
        feed_dict[self.in_neigh_mask] = batch.in_neigh_mask

        feed_dict[self.out_neigh_indices] = batch.out_neigh_indices
        feed_dict[self.out_neigh_edges] = batch.out_neigh_edges
        feed_dict[self.out_neigh_mask] = batch.out_neigh_mask

        feed_dict[self.entity_indices] = batch.entity_indices
        feed_dict[self.entity_indices_mask] = batch.entity_indices_mask
        feed_dict[self.refs] = batch.y
        if is_train:
            return sess.run([self.accu, self.loss, self.train_op], feed_dict)
        else:
            return sess.run([self.accu, self.loss, self.output], feed_dict)


In [0]:
!rm '/usr/local/lib/python2.7/dist-packages/pythonrouge/RELEASE-1.5.5/data/WordNet-2.0.exc.db'
!rm '/usr/local/lib/python2.7/dist-packages/pythonrouge/RELEASE-1.5.5/data/WordNet-2.0-Exceptions/WordNet-2.0.exc.db'
!cpan XML::DOM
!perl '/usr/local/lib/python2.7/dist-packages/pythonrouge/RELEASE-1.5.5/data/WordNet-2.0-Exceptions/buildExeptionDB.pl' '/usr/local/lib/python2.7/dist-packages/pythonrouge/RELEASE-1.5.5/data/WordNet-2.0-Exceptions' '/usr/local/lib/python2.7/dist-packages/pythonrouge/RELEASE-1.5.5/data/smart_common_words.txt' '/usr/local/lib/python2.7/dist-packages/pythonrouge/RELEASE-1.5.5/data/WordNet-2.0.exc.db'

In [0]:
!pip uninstall pythonrouge
!pip install git+https://github.com/tagucci/pythonrouge.git

Uninstalling pythonrouge-0.2:
  Would remove:
    /usr/local/lib/python2.7/dist-packages/pythonrouge-0.2.dist-info/*
    /usr/local/lib/python2.7/dist-packages/pythonrouge/*
Proceed (y/n)? y
  Successfully uninstalled pythonrouge-0.2
  Cloning https://github.com/tagucci/pythonrouge.git to /tmp/pip-req-build-Xd9aQ9
  Running command git clone -q https://github.com/tagucci/pythonrouge.git /tmp/pip-req-build-Xd9aQ9
  Stored in directory: /tmp/pip-ephem-wheel-cache-5l463C/wheels/fd/ff/be/6716935d513fa8656ab185cb0aa70aed382b72dda42bf09c95
Successfully built pythonrouge


In [0]:
def read_text_file(text_file):
    lines = []
    with open(text_file, "rt") as f:
        for line in f:
            line = line.decode('utf-8')
            lines.append(line.strip())
    return lines

def read_nary_file(inpath, options):
    all_words = []
    all_lemmas = []
    all_poses = []
    all_in_neigh = []
    all_in_label = []
    all_out_neigh = [] # [batch, node, neigh]
    all_out_label = [] # [batch, node, neigh]
    all_entity_indices = [] # [batch, 3, entity_size]
    all_y = []
    if options.class_num == 2:
        relation_set = {'resistance or non-response':0, 'sensitivity':0, 'response':0, 'resistance':0, 'None':1, }
    elif options.class_num == 5:
        relation_set = {'resistance or non-response':0, 'sensitivity':1, 'response':2, 'resistance':3, 'None':4, }
    else:
        assert False, 'Illegal class num'
    max_words = 0
    max_in_neigh = 0
    max_out_neigh = 0
    max_entity_size = 0
    with codecs.open(inpath, 'rU', 'utf-8') as f:
        for inst in json.load(f):
            words = []
            lemmas = []
            poses = []
            if options.only_single_sent and len(inst['sentences']) > 1:
                continue
            for sentence in inst['sentences']:
                for node in sentence['nodes']:
                    words.append(node['label'])
                    lemmas.append(node['lemma'])
                    poses.append(node['postag'])
            max_words = max(max_words, len(words))
            all_words.append(words)
            all_lemmas.append(lemmas)
            all_poses.append(poses)
            in_neigh = [[i,] for i,_ in enumerate(words)]
            in_label = [['self',] for i,_ in enumerate(words)]
            out_neigh = [[i,] for i,_ in enumerate(words)]
            out_label = [['self',] for i,_ in enumerate(words)]
            for sentence in inst['sentences']:
                for node in sentence['nodes']:
                    i = node['index']
                    for arc in node['arcs']:
                        j = arc['toIndex']
                        l = arc['label']
                        l = l.split('::')[0]
                        l = l.split('_')[0]
                        l = l.split('(')[0]
                        if j == -1 or l == '':
                            continue
                        in_neigh[j].append(i)
                        in_label[j].append(l)
                        out_neigh[i].append(j)
                        out_label[i].append(l)
            for _i in in_neigh:
                max_in_neigh = max(max_in_neigh, len(_i))
            for _o in out_neigh:
                max_out_neigh = max(max_out_neigh, len(_o))
            all_in_neigh.append(in_neigh)
            all_in_label.append(in_label)
            all_out_neigh.append(out_neigh)
            all_out_label.append(out_label)
            entity_indices = []
            for entity in inst['entities']:
                entity_indices.append(entity['indices'])
                max_entity_size = max(max_entity_size, len(entity['indices']))
            assert len(entity_indices) == options.entity_num
            all_entity_indices.append(entity_indices)
            all_y.append(relation_set[inst['relationLabel'].strip()])
    all_lex = all_lemmas if options.word_format == 'lemma' else all_words
    return zip(all_lex, all_poses, all_in_neigh, all_in_label, all_out_neigh, all_out_label, all_entity_indices, all_y), \
            max_words, max_in_neigh, max_out_neigh, max_entity_size


def read_nary_from_fof(fofpath, options):
    all_paths = read_text_file(fofpath)
    all_instances = []
    max_words = 0
    max_in_neigh = 0
    max_out_neigh = 0
    max_entity_size = 0
    for cur_path in all_paths:
        print(cur_path)
        cur_instances, cur_words, cur_in_neigh, cur_out_neigh, cur_entity_size = read_nary_file(cur_path, options)
        all_instances.extend(cur_instances)
        max_words = max(max_words, cur_words)
        max_in_neigh = max(max_in_neigh, cur_in_neigh)
        max_out_neigh = max(max_out_neigh, cur_out_neigh)
        max_entity_size = max(max_entity_size, cur_entity_size)
    return all_instances, max_words, max_in_neigh, max_out_neigh, max_entity_size


def collect_vocabs(all_instances):
    all_words = set()
    all_chars = set()
    all_edgelabels = set()
    for (lex, poses, in_neigh, in_label, out_neigh, out_label, entity_indices, y) in all_instances:
        all_words.update(lex)
        for l in lex:
            if l.isspace() == False: all_chars.update(l)
        for edges in in_label:
            all_edgelabels.update(edges)
        for edges in out_label:
            all_edgelabels.update(edges)
    return (all_words, all_chars, all_edgelabels)

class G2SDataStream(object):
    def __init__(self, all_instances, word_vocab=None, char_vocab=None, edgelabel_vocab=None, options=None,
                 isShuffle=False, isLoop=False, isSort=True, batch_size=-1):
        self.options = options
        if batch_size ==-1: batch_size=options.batch_size
        # index tokens and filter the dataset
        instances = []
        for (lex, poses, in_neigh, in_label, out_neigh, out_label, entity_indices, y) in all_instances:
            if options.max_node_num != -1 and len(lex) > options.max_node_num:
                continue # remove very long passages
            in_neigh = [x[:options.max_in_neigh_num] for x in in_neigh]
            in_label = [x[:options.max_in_neigh_num] for x in in_label]
            out_neigh = [x[:options.max_out_neigh_num] for x in out_neigh]
            out_label = [x[:options.max_out_neigh_num] for x in out_label]

            lex_idx = word_vocab.to_index_sequence_for_list(lex)
            lex_chars_idx = None
            if options.with_char:
                lex_chars_idx = char_vocab.to_character_matrix_for_list(lex, max_char_per_word=options.max_char_per_word)
            in_label_idx = [edgelabel_vocab.to_index_sequence_for_list(edges) for edges in in_label]
            out_label_idx = [edgelabel_vocab.to_index_sequence_for_list(edges) for edges in out_label]
            instances.append((lex_idx, lex_chars_idx, in_neigh, in_label_idx, out_neigh, out_label_idx, entity_indices, y))

        all_instances = instances
        instances = None

        # sort instances based on length
        if isSort:
            all_instances = sorted(all_instances, key=lambda inst: len(inst[0]))
        if isShuffle:
            random.shuffle(all_instances)
            random.shuffle(all_instances)
        self.num_instances = len(all_instances)

        # distribute questions into different buckets
        batch_spans = make_batches(self.num_instances, batch_size)
        self.batches = []
        for batch_index, (batch_start, batch_end) in enumerate(batch_spans):
            cur_instances = []
            for i in xrange(batch_start, batch_end):
                cur_instances.append(all_instances[i])
            cur_batch = G2SBatch(cur_instances, options, word_vocab=word_vocab)
            self.batches.append(cur_batch)

        self.num_batch = len(self.batches)
        self.index_array = np.arange(self.num_batch)
        self.isShuffle = isShuffle
        if self.isShuffle: np.random.shuffle(self.index_array)
        self.isLoop = isLoop
        self.cur_pointer = 0

    def nextBatch(self):
        if self.cur_pointer>=self.num_batch:
            if not self.isLoop: return None
            self.cur_pointer = 0
            if self.isShuffle: np.random.shuffle(self.index_array)
        cur_batch = self.batches[self.index_array[self.cur_pointer]]
        self.cur_pointer += 1
        return cur_batch

    def reset(self):
        if self.isShuffle: np.random.shuffle(self.index_array)
        self.cur_pointer = 0

    def get_num_batch(self):
        return self.num_batch

    def get_num_instance(self):
        return self.num_instances

    def get_batch(self, i):
        if i>= self.num_batch: return None
        return self.batches[i]

class G2SBatch(object):
    def __init__(self, instances, options, word_vocab=None):
        self.options = options

        self.instances = instances # list of tuples
        self.batch_size = len(instances)
        self.vocab = word_vocab

        # node num
        self.node_num = [] # [batch_size]
        for (lex_idx, lex_chars_idx, in_neigh, in_label_idx, out_neigh, out_label_idx, entity_indices, y) in instances:
            self.node_num.append(len(lex_idx))
        self.node_num = np.array(self.node_num, dtype=np.int32)

        # node char num
        if options.with_char:
            self.nodes_chars_num = [[len(lex_chars_idx) for lex_chars_idx in instance[1]] for instance in instances]
            self.nodes_chars_num = pad_2d_vals_no_size(self.nodes_chars_num)

        # neigh mask
        self.in_neigh_mask = [] # [batch_size, node_num, neigh_num]
        self.out_neigh_mask = []
        self.entity_indices_mask = []
        for instance in instances:
            ins = []
            for in_neighs in instance[2]:
                ins.append([1 for _ in in_neighs])
            self.in_neigh_mask.append(ins)
            outs = []
            for out_neighs in instance[4]:
                outs.append([1 for _ in out_neighs])
            self.out_neigh_mask.append(outs)
            idxs = []
            for entity_indices in instance[6]:
                idxs.append([1 for _ in entity_indices])
            self.entity_indices_mask.append(idxs)
        self.in_neigh_mask = pad_3d_vals_no_size(self.in_neigh_mask)
        self.out_neigh_mask = pad_3d_vals_no_size(self.out_neigh_mask)
        self.entity_indices_mask = pad_3d_vals_no_size(self.entity_indices_mask)

        # the actual contents
        self.nodes = [x[0] for x in instances]
        if options.with_char:
            self.nodes_chars = [x[1] for x in instances] # [batch_size, sent_len, char_num]
        self.in_neigh_indices = [x[2] for x in instances]
        self.in_neigh_edges = [x[3] for x in instances]
        self.out_neigh_indices = [x[4] for x in instances]
        self.out_neigh_edges = [x[5] for x in instances]
        self.entity_indices = [x[6] for x in instances]
        self.y = [x[7] for x in instances]

        # making ndarray
        self.nodes = pad_2d_vals_no_size(self.nodes)
        if options.with_char:
            self.nodes_chars = pad_3d_vals_no_size(self.nodes_chars)
        self.in_neigh_indices = pad_3d_vals_no_size(self.in_neigh_indices)
        self.in_neigh_edges = pad_3d_vals_no_size(self.in_neigh_edges)
        self.out_neigh_indices = pad_3d_vals_no_size(self.out_neigh_indices)
        self.out_neigh_edges = pad_3d_vals_no_size(self.out_neigh_edges)
        self.entity_indices = pad_3d_vals_no_size(self.entity_indices)
        self.y = np.asarray(self.y, dtype='int32')

        assert self.in_neigh_mask.shape == self.in_neigh_indices.shape
        assert self.in_neigh_mask.shape == self.in_neigh_edges.shape
        assert self.out_neigh_mask.shape == self.out_neigh_indices.shape
        assert self.out_neigh_mask.shape == self.out_neigh_edges.shape
        assert self.entity_indices_mask.shape == self.entity_indices.shape

        assert self.entity_indices.shape[1] == options.entity_num
        assert self.entity_indices_mask.shape[1] == options.entity_num

    def get_amrside_anonyids(self, anony_ids):
        assert self.batch_size == 1 # only for beam search
        if self.options.__dict__.has_key("enc_word_vec_path"):
            assert self.options.enc_word_vec_path == self.options.dec_word_vec_path # only when enc_vocab == dec_vocab
        self.amr_anony_ids = set(self.instances[0][0]) & anony_ids # sent1 of inst_0


#if __name__ == "__main__":
    """all_instances, max_node_num, max_in_neigh_num, max_out_neigh_num, max_entity_size = read_nary_from_fof(
            './data/data_list', 'lemma')
    print sum(len(x[0]) for x in all_instances)/len(all_instances)
    print(max_in_neigh_num)
    print(max_out_neigh_num)
    print(max_node_num)
    print(max_entity_size)
    print('DONE!')"""

In [23]:
cc = SmoothingFunction()
FLAGS = None

def get_machine_name():
    return platform.node()

def vec2string(val):
    result = ""
    for v in val:
        result += " {}".format(v)
    return result.strip()


def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()


def document_bleu(vocab, gen, ref, suffix=''):
    genlex = [vocab.getLexical(x)[1] for x in gen]
    reflex = [[vocab.getLexical(x)[1],] for x in ref]
    genlst = [x.split() for x in genlex]
    reflst = [[x[0].split()] for x in reflex]
    f = codecs.open('gen.txt'+suffix,'w','utf-8')
    for line in genlex:
        print(line, end='\n', file=f)
    f.close()
    f = codecs.open('ref.txt'+suffix,'w','utf-8')
    for line in reflex:
        print(line[0], end='\n', file=f)
    f.close()
    return corpus_bleu(reflst, genlst, smoothing_function=cc.method3)


def evaluate(sess, valid_graph, devDataStream, options=None, suffix=''):
    devDataStream.reset()
    instances = []
    outputs = []
    dev_loss = 0.0
    dev_right = 0.0
    dev_total = 0.0
    for batch_index in xrange(devDataStream.get_num_batch()): # for each batch
        cur_batch = devDataStream.get_batch(batch_index)
        accu_value, loss_value, output_value = valid_graph.execute(sess, cur_batch, options, is_train=False)
        instances += cur_batch.instances
        outputs += output_value.flatten().tolist()
        dev_loss += loss_value
        dev_right += accu_value
        dev_total += cur_batch.batch_size

    return {'dev_loss':dev_loss, 'dev_accu':1.0*dev_right/dev_total, 'dev_right':dev_right, 'dev_total':dev_total, 'data':(instances,outputs)}


def main(_):
    print('Configurations:')
    print(FLAGS)

    log_dir = FLAGS.model_dir
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    path_prefix = log_dir + "/G2S.{}".format(FLAGS.suffix)
    log_file_path = path_prefix + ".log"
    print('Log file path: {}'.format(log_file_path))
    log_file = open(log_file_path, 'wt')
    log_file.write("{}\n".format(FLAGS))
    log_file.flush()

    # save configuration
    save_namespace(FLAGS, path_prefix + ".config.json")

    print('Loading train set.')
    if FLAGS.infile_format == 'fof':
        trainset, trn_node, trn_in_neigh, trn_out_neigh, trn_sent = read_nary_from_fof(FLAGS.train_path, FLAGS)
    else:
        trainset, trn_node, trn_in_neigh, trn_out_neigh, trn_sent = read_nary_file(FLAGS.train_path, FLAGS)

    random.shuffle(trainset)
    devset = trainset[:200]
    trainset = trainset[200:]

    print('Number of training samples: {}'.format(len(trainset)))
    print('Number of dev samples: {}'.format(len(devset)))

    max_node = trn_node
    max_in_neigh = trn_in_neigh
    max_out_neigh = trn_out_neigh
    max_sent = trn_sent
    print('Max node number: {}, while max allowed is {}'.format(max_node, FLAGS.max_node_num))
    print('Max parent number: {}, truncated to {}'.format(max_in_neigh, FLAGS.max_in_neigh_num))
    print('Max children number: {}, truncated to {}'.format(max_out_neigh, FLAGS.max_out_neigh_num))
    print('Max entity size: {}, truncated to {}'.format(max_sent, FLAGS.max_entity_size))

    word_vocab = None
    char_vocab = None
    edgelabel_vocab = None
    has_pretrained_model = False
    best_path = path_prefix + ".best.model"
    if os.path.exists(best_path + ".index"):
        has_pretrained_model = True
        print('!!Existing pretrained model. Loading vocabs.')
        word_vocab = Vocab(FLAGS.word_vec_path, fileformat='txt2')
        print('word_vocab: {}'.format(word_vocab.word_vecs.shape))
        char_vocab = None
        if FLAGS.with_char:
            char_vocab = Vocab(path_prefix + ".char_vocab", fileformat='txt2')
            print('char_vocab: {}'.format(char_vocab.word_vecs.shape))
        edgelabel_vocab = Vocab(path_prefix + ".edgelabel_vocab", fileformat='txt2')
    else:
        print('Collecting vocabs.')
        (allWords, allChars, allEdgelabels) = collect_vocabs(trainset)
        print('Number of words: {}'.format(len(allWords)))
        print('Number of allChars: {}'.format(len(allChars)))
        print('Number of allEdgelabels: {}'.format(len(allEdgelabels)))

        word_vocab = Vocab(FLAGS.word_vec_path, fileformat='txt2')
        char_vocab = None
        if FLAGS.with_char:
            char_vocab = Vocab(voc=allChars, dim=FLAGS.char_dim, fileformat='build')
            char_vocab.dump_to_txt2(path_prefix + ".char_vocab")
        edgelabel_vocab = Vocab(voc=allEdgelabels, dim=FLAGS.edgelabel_dim, fileformat='build')
        edgelabel_vocab.dump_to_txt2(path_prefix + ".edgelabel_vocab")

    print('word vocab size {}'.format(word_vocab.vocab_size))
    sys.stdout.flush()

    print('Build DataStream ... ')
    trainDataStream = G2SDataStream(trainset, word_vocab, char_vocab, edgelabel_vocab, options=FLAGS,
                 isShuffle=True, isLoop=True, isSort=False)

    devDataStream = G2SDataStream(devset, word_vocab, char_vocab, edgelabel_vocab, options=FLAGS,
                 isShuffle=False, isLoop=False, isSort=False)
    print('Number of instances in trainDataStream: {}'.format(trainDataStream.get_num_instance()))
    print('Number of instances in devDataStream: {}'.format(devDataStream.get_num_instance()))
    print('Number of batches in trainDataStream: {}'.format(trainDataStream.get_num_batch()))
    print('Number of batches in devDataStream: {}'.format(devDataStream.get_num_batch()))
    sys.stdout.flush()

    # initialize the best bleu and accu scores for current training session
    best_accu = FLAGS.best_accu if FLAGS.__dict__.has_key('best_accu') else 0.0
    if best_accu > 0.0:
        print('With initial dev accuracy {}'.format(best_accu))

    init_scale = 0.01
    with tf.Graph().as_default():
        initializer = tf.random_uniform_initializer(-init_scale, init_scale)
        with tf.name_scope("Train"):
            with tf.variable_scope("Model", reuse=None, initializer=initializer):
                train_graph = ModelGraph(word_vocab=word_vocab, Edgelabel_vocab=edgelabel_vocab,
                                         char_vocab=char_vocab, options=FLAGS, mode='train')

        with tf.name_scope("Valid"):
            with tf.variable_scope("Model", reuse=True, initializer=initializer):
                valid_graph = ModelGraph(word_vocab=word_vocab, Edgelabel_vocab=edgelabel_vocab,
                                         char_vocab=char_vocab, options=FLAGS, mode='evaluate')

        initializer = tf.global_variables_initializer()

        vars_ = {}
        for var in tf.all_variables():
            if "word_embedding" in var.name: continue
            if not var.name.startswith("Model"): continue
            vars_[var.name.split(":")[0]] = var
        saver = tf.train.Saver(vars_)

        sess = tf.Session()
        sess.run(initializer)
        if has_pretrained_model:
            print("Restoring model from " + best_path)
            saver.restore(sess, best_path)
            print("DONE!")

            if abs(best_accu) < 0.00001:
                print("Getting ACCU score for the model")
                best_accu = evaluate(sess, valid_graph, devDataStream, options=FLAGS)['dev_accu']
                FLAGS.best_accu = best_accu
                save_namespace(FLAGS, path_prefix + ".config.json")
                print('ACCU = %.4f' % best_accu)
                log_file.write('ACCU = %.4f\n' % best_accu)

        print('Start the training loop.')
        train_size = trainDataStream.get_num_batch()
        max_steps = train_size * FLAGS.max_epochs
        last_step = 0
        total_loss = 0.0
        start_time = time.time()
        for step in xrange(max_steps):
            cur_batch = trainDataStream.nextBatch()
            _, loss_value, _ = train_graph.execute(sess, cur_batch, FLAGS, is_train=True)
            total_loss += loss_value

            if step % 100==0:
                print('{} '.format(step), end="")
                sys.stdout.flush()

            # Save a checkpoint and evaluate the model periodically.
            if (step + 1) % trainDataStream.get_num_batch() == 0 or (step + 1) == max_steps:
                print()
                duration = time.time() - start_time
                print('Step %d: loss = %.2f (%.3f sec)' % (step, total_loss/(step-last_step), duration))
                log_file.write('Step %d: loss = %.2f (%.3f sec)\n' % (step, total_loss/(step-last_step), duration))
                sys.stdout.flush()
                log_file.flush()
                last_step = step
                total_loss = 0.0

                # Evaluate against the validation set.
                start_time = time.time()
                print('Validation Data Eval:')
                res_dict = evaluate(sess, valid_graph, devDataStream, options=FLAGS, suffix=str(step))
                dev_loss = res_dict['dev_loss']
                dev_accu = res_dict['dev_accu']
                dev_right = int(res_dict['dev_right'])
                dev_total = int(res_dict['dev_total'])
                print('Dev loss = %.4f' % dev_loss)
                log_file.write('Dev loss = %.4f\n' % dev_loss)
                print('Dev accu = %.4f %d/%d' % (dev_accu, dev_right, dev_total))
                log_file.write('Dev accu = %.4f %d/%d\n' % (dev_accu, dev_right, dev_total))
                log_file.flush()
                if best_accu < dev_accu:
                    print('Saving weights, ACCU {} (prev_best) < {} (cur)'.format(best_accu, dev_accu))
                    saver.save(sess, best_path)
                    best_accu = dev_accu
                    FLAGS.best_accu = dev_accu
                    save_namespace(FLAGS, path_prefix + ".config.json")
                    json.dump(res_dict['data'], open(FLAGS.output_path,'w'))
                duration = time.time() - start_time
                print('Duration %.3f sec' % (duration))
                sys.stdout.flush()

                log_file.write('Duration %.3f sec\n' % (duration))
                log_file.flush()
                print("(step + 1) == max_steps",step+1,max_steps)
        
    log_file.close()


def enrich_options(options):
    if not options.__dict__.has_key("infile_format"):
        options.__dict__["infile_format"] = "fof"

    return options


if __name__ == '__main__':
    
    parser = argparse.ArgumentParser()
    parser.add_argument('--config_path', type=str,nargs='?',default ='/content/drive/My Drive/nary-grn-master/gs_lstm/config.json', help='Configuration file.')
    
    #os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
    #os.environ["CUDA_VISIBLE_DEVICES"]="2"
    
    #print("CUDA_VISIBLE_DEVICES " + os.environ['CUDA_VISIBLE_DEVICES'])
    FLAGS, unparsed = parser.parse_known_args()

    print(FLAGS.config_path)
    unparsed=None
    if FLAGS.config_path is not None:
        print('Loading the configuration from ' + FLAGS.config_path)
        FLAGS = load_namespace(FLAGS.config_path)
    
    FLAGS = enrich_options(FLAGS)
    

    sys.stdout.flush()
    tf.app.run(main=main,argv=None)
    #If running through command line
    #tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

/content/drive/My Drive/nary-grn-master/gs_lstm/config.json
Loading the configuration from /content/drive/My Drive/nary-grn-master/gs_lstm/config.json
Configurations:
Log file path: logs/G2S.cross_mul_0.log
Loading train set.
/content/drive/My Drive/nary-grn-master/peng_data/data/drug_gene_var/1/data_graph_1
Number of training samples: 505
Number of dev samples: 200
Max node number: 343, while max allowed is 450
Max parent number: 34, truncated to 45
Max children number: 34, truncated to 45
Max entity size: 5, truncated to 5
Number of words: 2665
Number of allChars: 98
Number of allEdgelabels: 109
word vocab size 15736
Build DataStream ... 
Number of instances in trainDataStream: 505
Number of instances in devDataStream: 200
Number of batches in trainDataStream: 64
Number of batches in devDataStream: 25


/usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Return from here, just evaluate
Start the training loop.
0 
Step 63: loss = 1.52 (7.623 sec)
Validation Data Eval:
Dev loss = 31.0420
Dev accu = 0.3850 77/200
Saving weights, ACCU 0.0 (prev_best) < 0.385 (cur)
Duration 3.013 sec
(step + 1) == max_steps 64 3200
100 
Step 127: loss = 1.21 (8.224 sec)
Validation Data Eval:
Dev loss = 30.2684
Dev accu = 0.3850 77/200
Duration 0.881 sec
(step + 1) == max_steps 128 3200

Step 191: loss = 1.19 (6.049 sec)
Validation Data Eval:
Dev loss = 29.9925
Dev accu = 0.4000 80/200
Saving weights, ACCU 0.385 (prev_best) < 0.4 (cur)
Duration 1.929 sec
(step + 1) == max_steps 192 3200
200 
Step 255: loss = 1.18 (7.174 sec)
Validation Data Eval:
Dev loss = 29.8386
Dev accu = 0.4450 89/200
Saving weights, ACCU 0.4 (prev_best) < 0.445 (cur)
Duration 1.969 sec
(step + 1) == max_steps 256 3200
300 
Step 319: loss = 1.17 (7.151 sec)
Validation Data Eval:
Dev loss = 29.6396
Dev accu = 0.4050 81/200
Duration 0.966 sec
(step + 1) == max_steps 320 3200

Step 383: lo

SystemExit: ignored

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [0]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--model_prefix', type=str,default='/content/logs/G2S.cross_mul_0', help='Prefix to the models.')
    parser.add_argument('--in_path', type=str,  help='The path to the test file.',default='/content/drive/My Drive/nary-grn-master/gs_lstm/data/test_list_0')
    parser.add_argument('--out_path', type=str, help='The path to the test file.',default='/content/logs/results_c_b_0.json')

    args, unparsed = parser.parse_known_args()

    model_prefix = args.model_prefix
    in_path = args.in_path
    out_path = args.out_path

    #os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
    #os.environ["CUDA_VISIBLE_DEVICES"]="0"

    #print("CUDA_VISIBLE_DEVICES " + os.environ['CUDA_VISIBLE_DEVICES'])

    # load the configuration file
    print('Loading configurations from ' + model_prefix + ".config.json")
    FLAGS = load_namespace(model_prefix + ".config.json")
    FLAGS = enrich_options(FLAGS)

    # load vocabs
    print('Loading vocabs.')
    word_vocab = Vocab(FLAGS.word_vec_path, fileformat='txt2')
    print('word_vocab: {}'.format(word_vocab.word_vecs.shape))
    edgelabel_vocab = Vocab(model_prefix + ".edgelabel_vocab", fileformat='txt2')
    print('edgelabel_vocab: {}'.format(edgelabel_vocab.word_vecs.shape))
    char_vocab = None
    if FLAGS.with_char:
        char_vocab = Vocab(model_prefix + ".char_vocab", fileformat='txt2')
        print('char_vocab: {}'.format(char_vocab.word_vecs.shape))

    print('Loading test set from {}.'.format(in_path))
    if FLAGS.infile_format == 'fof':
        testset, _, _, _, _ = read_nary_from_fof(in_path, FLAGS)
    else:
        testset, _, _, _, _ = read_nary_file(in_path, FLAGS)
    print('Number of samples: {}'.format(len(testset)))

    print('Build DataStream ... ')
    batch_size=-1
    devDataStream = G2SDataStream(testset, word_vocab, char_vocab, edgelabel_vocab, options=FLAGS,
                 isShuffle=False, isLoop=False, isSort=False, batch_size=batch_size)
    print('Number of instances in testDataStream: {}'.format(devDataStream.get_num_instance()))
    print('Number of batches in testDataStream: {}'.format(devDataStream.get_num_batch()))

    best_path = model_prefix + ".best.model"
    with tf.Graph().as_default():
        initializer = tf.random_uniform_initializer(-0.01, 0.01)
        with tf.name_scope("Valid"):
            with tf.variable_scope("Model", reuse=False, initializer=initializer):
                valid_graph = ModelGraph(word_vocab=word_vocab, char_vocab=char_vocab, Edgelabel_vocab=edgelabel_vocab,
                                         options=FLAGS, mode="evaluate")

        ## remove word _embedding
        vars_ = {}
        for var in tf.all_variables():
            if "word_embedding" in var.name: continue
            if not var.name.startswith("Model"): continue
            vars_[var.name.split(":")[0]] = var
        saver = tf.train.Saver(vars_)

        initializer = tf.global_variables_initializer()
        sess = tf.Session()
        sess.run(initializer)

        saver.restore(sess, best_path) # restore the model

        devDataStream.reset()
        instances = []
        outputs = []
        test_loss = 0.0
        test_right = 0.0
        test_total = 0.0
        start_time = time.time()
        for batch_index in xrange(devDataStream.get_num_batch()): # for each batch
            cur_batch = devDataStream.get_batch(batch_index)
            accu_value, loss_value, output_value = valid_graph.execute(sess, cur_batch, FLAGS, is_train=False)
            instances += cur_batch.instances
            outputs += output_value.flatten().tolist()
            test_loss += loss_value
            test_right += accu_value
            test_total += cur_batch.batch_size
        duration = time.time() - start_time
        print('Decoding time %.3f sec' % (duration))

        assert len(instances) == len(outputs)
        json.dump((instances,outputs,testset), open(out_path,'w'))

        print('Test accu {}, right {}, total {}'.format(1.0*test_right/test_total, test_right, test_total))

Loading configurations from /content/logs/G2S.cross_mul_0.config.json
Loading vocabs.
word_vocab: (15736, 100)
edgelabel_vocab: (111, 3)
Loading test set from /content/drive/My Drive/nary-grn-master/gs_lstm/data/test_list_0.
/content/drive/My Drive/nary-grn-master/peng_data/data/drug_gene_var/0/data_graph_1
Number of samples: 758
Build DataStream ... 
Number of instances in testDataStream: 758
Number of batches in testDataStream: 95
Return from here, just evaluate
Decoding time 2.938 sec
Test accu 0.627968337731, right 476.0, total 758.0


In [22]:
#import abc
import RelationExtraction

#class RelationExtraction1(RelationExtraction):
class RelationExtraction1:
  cc = SmoothingFunction()
  FLAGS = None

  def get_machine_name():
      return platform.node()

  def vec2string(val):
      result = ""
      for v in val:
          result += " {}".format(v)
      return result.strip()


  def softmax(x):
      """Compute softmax values for each sets of scores in x."""
      e_x = np.exp(x - np.max(x))
      return e_x / e_x.sum()


  def document_bleu(vocab, gen, ref, suffix=''):
      genlex = [vocab.getLexical(x)[1] for x in gen]
      reflex = [[vocab.getLexical(x)[1],] for x in ref]
      genlst = [x.split() for x in genlex]
      reflst = [[x[0].split()] for x in reflex]
      f = codecs.open('gen.txt'+suffix,'w','utf-8')
      for line in genlex:
          print(line, end='\n', file=f)
      f.close()
      f = codecs.open('ref.txt'+suffix,'w','utf-8')
      for line in reflex:
          print(line[0], end='\n', file=f)
      f.close()
      return corpus_bleu(reflst, genlst, smoothing_function=cc.method3)


  def evaluate(sess, valid_graph, devDataStream, options=None, suffix=''):
      devDataStream.reset()
      instances = []
      outputs = []
      dev_loss = 0.0
      dev_right = 0.0
      dev_total = 0.0
      for batch_index in xrange(devDataStream.get_num_batch()): # for each batch
          cur_batch = devDataStream.get_batch(batch_index)
          accu_value, loss_value, output_value = valid_graph.execute(sess, cur_batch, options, is_train=False)
          instances += cur_batch.instances
          outputs += output_value.flatten().tolist()
          dev_loss += loss_value
          dev_right += accu_value
          dev_total += cur_batch.batch_size

      return {'dev_loss':dev_loss, 'dev_accu':1.0*dev_right/dev_total, 'dev_right':dev_right, 'dev_total':dev_total, 'data':(instances,outputs)}


  def main(_):
      print('Configurations:')
      print(FLAGS)

      log_dir = FLAGS.model_dir
      if not os.path.exists(log_dir):
          os.makedirs(log_dir)

      path_prefix = log_dir + "/G2S.{}".format(FLAGS.suffix)
      log_file_path = path_prefix + ".log"
      print('Log file path: {}'.format(log_file_path))
      log_file = open(log_file_path, 'wt')
      log_file.write("{}\n".format(FLAGS))
      log_file.flush()

      # save configuration
      save_namespace(FLAGS, path_prefix + ".config.json")

      print('Loading train set.')
      if FLAGS.infile_format == 'fof':
          trainset, trn_node, trn_in_neigh, trn_out_neigh, trn_sent = read_nary_from_fof(FLAGS.train_path, FLAGS)
      else:
          trainset, trn_node, trn_in_neigh, trn_out_neigh, trn_sent = read_nary_file(FLAGS.train_path, FLAGS)

      random.shuffle(trainset)
      devset = trainset[:200]
      trainset = trainset[200:]

      print('Number of training samples: {}'.format(len(trainset)))
      print('Number of dev samples: {}'.format(len(devset)))

      max_node = trn_node
      max_in_neigh = trn_in_neigh
      max_out_neigh = trn_out_neigh
      max_sent = trn_sent
      print('Max node number: {}, while max allowed is {}'.format(max_node, FLAGS.max_node_num))
      print('Max parent number: {}, truncated to {}'.format(max_in_neigh, FLAGS.max_in_neigh_num))
      print('Max children number: {}, truncated to {}'.format(max_out_neigh, FLAGS.max_out_neigh_num))
      print('Max entity size: {}, truncated to {}'.format(max_sent, FLAGS.max_entity_size))

      word_vocab = None
      char_vocab = None
      edgelabel_vocab = None
      has_pretrained_model = False
      best_path = path_prefix + ".best.model"
      if os.path.exists(best_path + ".index"):
          has_pretrained_model = True
          print('!!Existing pretrained model. Loading vocabs.')
          word_vocab = Vocab(FLAGS.word_vec_path, fileformat='txt2')
          print('word_vocab: {}'.format(word_vocab.word_vecs.shape))
          char_vocab = None
          if FLAGS.with_char:
              char_vocab = Vocab(path_prefix + ".char_vocab", fileformat='txt2')
              print('char_vocab: {}'.format(char_vocab.word_vecs.shape))
          edgelabel_vocab = Vocab(path_prefix + ".edgelabel_vocab", fileformat='txt2')
      else:
          print('Collecting vocabs.')
          (allWords, allChars, allEdgelabels) = collect_vocabs(trainset)
          print('Number of words: {}'.format(len(allWords)))
          print('Number of allChars: {}'.format(len(allChars)))
          print('Number of allEdgelabels: {}'.format(len(allEdgelabels)))

          word_vocab = Vocab(FLAGS.word_vec_path, fileformat='txt2')
          char_vocab = None
          if FLAGS.with_char:
              char_vocab = Vocab(voc=allChars, dim=FLAGS.char_dim, fileformat='build')
              char_vocab.dump_to_txt2(path_prefix + ".char_vocab")
          edgelabel_vocab = Vocab(voc=allEdgelabels, dim=FLAGS.edgelabel_dim, fileformat='build')
          edgelabel_vocab.dump_to_txt2(path_prefix + ".edgelabel_vocab")

      print('word vocab size {}'.format(word_vocab.vocab_size))
      sys.stdout.flush()

      print('Build DataStream ... ')
      trainDataStream = G2SDataStream(trainset, word_vocab, char_vocab, edgelabel_vocab, options=FLAGS,
                   isShuffle=True, isLoop=True, isSort=False)

      devDataStream = G2SDataStream(devset, word_vocab, char_vocab, edgelabel_vocab, options=FLAGS,
                   isShuffle=False, isLoop=False, isSort=False)
      print('Number of instances in trainDataStream: {}'.format(trainDataStream.get_num_instance()))
      print('Number of instances in devDataStream: {}'.format(devDataStream.get_num_instance()))
      print('Number of batches in trainDataStream: {}'.format(trainDataStream.get_num_batch()))
      print('Number of batches in devDataStream: {}'.format(devDataStream.get_num_batch()))
      sys.stdout.flush()

      # initialize the best bleu and accu scores for current training session
      best_accu = FLAGS.best_accu if FLAGS.__dict__.has_key('best_accu') else 0.0
      if best_accu > 0.0:
          print('With initial dev accuracy {}'.format(best_accu))

      init_scale = 0.01
      with tf.Graph().as_default():
          initializer = tf.random_uniform_initializer(-init_scale, init_scale)
          with tf.name_scope("Train"):
              with tf.variable_scope("Model", reuse=None, initializer=initializer):
                  train_graph = ModelGraph(word_vocab=word_vocab, Edgelabel_vocab=edgelabel_vocab,
                                           char_vocab=char_vocab, options=FLAGS, mode='train')

          with tf.name_scope("Valid"):
              with tf.variable_scope("Model", reuse=True, initializer=initializer):
                  valid_graph = ModelGraph(word_vocab=word_vocab, Edgelabel_vocab=edgelabel_vocab,
                                           char_vocab=char_vocab, options=FLAGS, mode='evaluate')

          initializer = tf.global_variables_initializer()

          vars_ = {}
          for var in tf.all_variables():
              if "word_embedding" in var.name: continue
              if not var.name.startswith("Model"): continue
              vars_[var.name.split(":")[0]] = var
          saver = tf.train.Saver(vars_)

          sess = tf.Session()
          sess.run(initializer)
          if has_pretrained_model:
              print("Restoring model from " + best_path)
              saver.restore(sess, best_path)
              print("DONE!")

              if abs(best_accu) < 0.00001:
                  print("Getting ACCU score for the model")
                  best_accu = evaluate(sess, valid_graph, devDataStream, options=FLAGS)['dev_accu']
                  FLAGS.best_accu = best_accu
                  save_namespace(FLAGS, path_prefix + ".config.json")
                  print('ACCU = %.4f' % best_accu)
                  log_file.write('ACCU = %.4f\n' % best_accu)

          print('Start the training loop.')
          train_size = trainDataStream.get_num_batch()
          max_steps = train_size * FLAGS.max_epochs
          last_step = 0
          total_loss = 0.0
          start_time = time.time()
          for step in xrange(max_steps):
              cur_batch = trainDataStream.nextBatch()
              _, loss_value, _ = train_graph.execute(sess, cur_batch, FLAGS, is_train=True)
              total_loss += loss_value

              if step % 100==0:
                  print('{} '.format(step), end="")
                  sys.stdout.flush()

              # Save a checkpoint and evaluate the model periodically.
              if (step + 1) % trainDataStream.get_num_batch() == 0 or (step + 1) == max_steps:
                  print()
                  duration = time.time() - start_time
                  print('Step %d: loss = %.2f (%.3f sec)' % (step, total_loss/(step-last_step), duration))
                  log_file.write('Step %d: loss = %.2f (%.3f sec)\n' % (step, total_loss/(step-last_step), duration))
                  sys.stdout.flush()
                  log_file.flush()
                  last_step = step
                  total_loss = 0.0

                  # Evaluate against the validation set.
                  start_time = time.time()
                  print('Validation Data Eval:')
                  res_dict = evaluate(sess, valid_graph, devDataStream, options=FLAGS, suffix=str(step))
                  dev_loss = res_dict['dev_loss']
                  dev_accu = res_dict['dev_accu']
                  dev_right = int(res_dict['dev_right'])
                  dev_total = int(res_dict['dev_total'])
                  print('Dev loss = %.4f' % dev_loss)
                  log_file.write('Dev loss = %.4f\n' % dev_loss)
                  print('Dev accu = %.4f %d/%d' % (dev_accu, dev_right, dev_total))
                  log_file.write('Dev accu = %.4f %d/%d\n' % (dev_accu, dev_right, dev_total))
                  log_file.flush()
                  if best_accu < dev_accu:
                      print('Saving weights, ACCU {} (prev_best) < {} (cur)'.format(best_accu, dev_accu))
                      saver.save(sess, best_path)
                      best_accu = dev_accu
                      FLAGS.best_accu = dev_accu
                      save_namespace(FLAGS, path_prefix + ".config.json")
                      json.dump(res_dict['data'], open(FLAGS.output_path,'w'))
                  duration = time.time() - start_time
                  print('Duration %.3f sec' % (duration))
                  sys.stdout.flush()

                  log_file.write('Duration %.3f sec\n' % (duration))
                  log_file.flush()
                  print("(step + 1) == max_steps",step+1,max_steps)

      log_file.close()


  def enrich_options(options):
      if not options.__dict__.has_key("infile_format"):
          options.__dict__["infile_format"] = "fof"

      return options

  def __init__(self):
    pass
  
  def predict(self):
    parser = argparse.ArgumentParser()
    parser.add_argument('--model_prefix', type=str,default='/content/logs/G2S.cross_mul_0', help='Prefix to the models.')
    parser.add_argument('--in_path', type=str,  help='The path to the test file.',default='/content/drive/My Drive/nary-grn-master/gs_lstm/data/test_list_0')
    parser.add_argument('--out_path', type=str, help='The path to the test file.',default='/content/logs/results_c_b_0.json')

    args, unparsed = parser.parse_known_args()

    model_prefix = args.model_prefix
    in_path = args.in_path
    out_path = args.out_path

    #os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
    #os.environ["CUDA_VISIBLE_DEVICES"]="0"

    #print("CUDA_VISIBLE_DEVICES " + os.environ['CUDA_VISIBLE_DEVICES'])

    # load the configuration file
    print('Loading configurations from ' + model_prefix + ".config.json")
    FLAGS = load_namespace(model_prefix + ".config.json")
    FLAGS = enrich_options(FLAGS)

    # load vocabs
    print('Loading vocabs.')
    word_vocab = Vocab(FLAGS.word_vec_path, fileformat='txt2')
    print('word_vocab: {}'.format(word_vocab.word_vecs.shape))
    edgelabel_vocab = Vocab(model_prefix + ".edgelabel_vocab", fileformat='txt2')
    print('edgelabel_vocab: {}'.format(edgelabel_vocab.word_vecs.shape))
    char_vocab = None
    if FLAGS.with_char:
        char_vocab = Vocab(model_prefix + ".char_vocab", fileformat='txt2')
        print('char_vocab: {}'.format(char_vocab.word_vecs.shape))

    print('Loading test set from {}.'.format(in_path))
    if FLAGS.infile_format == 'fof':
        testset, _, _, _, _ = read_nary_from_fof(in_path, FLAGS)
    else:
        testset, _, _, _, _ = read_nary_file(in_path, FLAGS)
    print('Number of samples: {}'.format(len(testset)))

    print('Build DataStream ... ')
    batch_size=-1
    devDataStream = G2SDataStream(testset, word_vocab, char_vocab, edgelabel_vocab, options=FLAGS,
                 isShuffle=False, isLoop=False, isSort=False, batch_size=batch_size)
    print('Number of instances in testDataStream: {}'.format(devDataStream.get_num_instance()))
    print('Number of batches in testDataStream: {}'.format(devDataStream.get_num_batch()))

    best_path = model_prefix + ".best.model"
    with tf.Graph().as_default():
        initializer = tf.random_uniform_initializer(-0.01, 0.01)
        with tf.name_scope("Valid"):
            with tf.variable_scope("Model", reuse=False, initializer=initializer):
                valid_graph = ModelGraph(word_vocab=word_vocab, char_vocab=char_vocab, Edgelabel_vocab=edgelabel_vocab,
                                         options=FLAGS, mode="evaluate")

        ## remove word _embedding
        vars_ = {}
        for var in tf.all_variables():
            if "word_embedding" in var.name: continue
            if not var.name.startswith("Model"): continue
            vars_[var.name.split(":")[0]] = var
        saver = tf.train.Saver(vars_)

        initializer = tf.global_variables_initializer()
        sess = tf.Session()
        sess.run(initializer)

        saver.restore(sess, best_path) # restore the model

        devDataStream.reset()
        instances = []
        outputs = []
        test_loss = 0.0
        test_right = 0.0
        test_total = 0.0
        start_time = time.time()
        for batch_index in xrange(devDataStream.get_num_batch()): # for each batch
            cur_batch = devDataStream.get_batch(batch_index)
            accu_value, loss_value, output_value = valid_graph.execute(sess, cur_batch, FLAGS, is_train=False)
            instances += cur_batch.instances
            outputs += output_value.flatten().tolist()
            test_loss += loss_value
            test_right += accu_value
            test_total += cur_batch.batch_size
        duration = time.time() - start_time
        print('Decoding time %.3f sec' % (duration))

        assert len(instances) == len(outputs)
        json.dump((instances,outputs,testset), open(out_path,'w'))

        print('Test accu {}, right {}, total {}'.format(1.0*test_right/test_total, test_right, test_total))
    pass
  
  def train(self):
    parser = argparse.ArgumentParser()
    parser.add_argument('--config_path', type=str,nargs='?',default ='/content/drive/My Drive/nary-grn-master/gs_lstm/config.json', help='Configuration file.')
    
    #os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
    #os.environ["CUDA_VISIBLE_DEVICES"]="2"
    
    #print("CUDA_VISIBLE_DEVICES " + os.environ['CUDA_VISIBLE_DEVICES'])
    FLAGS, unparsed = parser.parse_known_args()

    print(FLAGS.config_path)
    unparsed=None
    if FLAGS.config_path is not None:
        print('Loading the configuration from ' + FLAGS.config_path)
        FLAGS = load_namespace(FLAGS.config_path)
    
    
    FLAGS = enrich_options(FLAGS)
    

    sys.stdout.flush()
    tf.app.run()
    #If running through command line
    #tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)
if __name__ == '__main__': 
  x=RelationExtraction1()
  x.train()
  x.predict()
    

/content/drive/My Drive/nary-grn-master/gs_lstm/config.json
Loading the configuration from /content/drive/My Drive/nary-grn-master/gs_lstm/config.json


NameError: ignored

In [13]:
x=RelationExtraction1()


Loading configurations from /content/logs/G2S.cross_mul_0.config.json


IOError: ignored